## Django Testing

Testing is a crucial part of Django development that ensures your application works correctly and prevents regressions. Django provides a comprehensive testing framework built on Python's unittest module with additional features for web development.

### Why Test?
- **Catch bugs early**: Prevent issues before they reach production
- **Refactor with confidence**: Ensure changes don't break existing functionality
- **Document behavior**: Tests serve as living documentation
- **Improve design**: Writing tests often reveals design issues
- **Facilitate collaboration**: Tests help new developers understand the codebase

### Django Testing Framework

Django's testing framework includes:
- **TestCase**: Base class for all Django tests
- **TransactionTestCase**: For tests requiring transaction management
- **LiveServerTestCase**: For tests requiring a live server
- **Test fixtures**: Pre-loaded data for tests
- **Test client**: Simulate HTTP requests
- **Assertion methods**: Specialized assertions for Django objects

### Test Structure
```
myapp/
    tests/
        __init__.py
        test_models.py
        test_views.py
        test_forms.py
        test_api.py
        fixtures/
            test_data.json
```

### Running Tests
```bash
# Run all tests
python manage.py test

# Run specific app tests
python manage.py test myapp

# Run specific test class
python manage.py test myapp.tests.TestClass

# Run specific test method
python manage.py test myapp.tests.TestClass.test_method

# Run with verbosity
python manage.py test --verbosity=2

# Run with coverage
coverage run manage.py test
coverage report
```

## Unit Testing

Unit tests focus on testing individual components in isolation. In Django, this typically means testing models, utilities, and business logic.

### Basic Test Structure

#### Test Case Setup
```python
# tests/test_models.py
from django.test import TestCase
from django.utils import timezone
from myapp.models import Article, Author

class ArticleModelTest(TestCase):
    def setUp(self):
        """Set up test data before each test method"""
        self.author = Author.objects.create(
            name="Test Author",
            email="author@example.com"
        )
        self.article = Article.objects.create(
            title="Test Article",
            content="Test content",
            author=self.author,
            pub_date=timezone.now()
        )
    
    def tearDown(self):
        """Clean up after each test method"""
        pass
    
    def test_article_creation(self):
        """Test that an article can be created"""
        self.assertEqual(self.article.title, "Test Article")
        self.assertEqual(self.article.author.name, "Test Author")
        self.assertIsNotNone(self.article.pub_date)
    
    def test_article_str_method(self):
        """Test the string representation of an article"""
        expected = f"{self.article.title} by {self.author.name}"
        self.assertEqual(str(self.article), expected)
```

### Model Method Testing

#### Testing Custom Model Methods
```python
# models.py
class Article(models.Model):
    title = models.CharField(max_length=200)
    content = models.TextField()
    pub_date = models.DateTimeField(default=timezone.now)
    status = models.CharField(
        max_length=10,
        choices=[('draft', 'Draft'), ('published', 'Published')],
        default='draft'
    )
    
    def is_published(self):
        return self.status == 'published' and self.pub_date <= timezone.now()
    
    def get_word_count(self):
        return len(self.content.split())
    
    def get_reading_time(self):
        words_per_minute = 200
        word_count = self.get_word_count()
        return max(1, round(word_count / words_per_minute))

# tests/test_models.py
class ArticleMethodTest(TestCase):
    def setUp(self):
        self.published_article = Article.objects.create(
            title="Published Article",
            content="This is a test article with some content.",
            status="published",
            pub_date=timezone.now() - timezone.timedelta(days=1)
        )
        self.draft_article = Article.objects.create(
            title="Draft Article",
            content="This is a draft article.",
            status="draft"
        )
    
    def test_is_published_true(self):
        """Test is_published returns True for published articles"""
        self.assertTrue(self.published_article.is_published())
    
    def test_is_published_false_for_draft(self):
        """Test is_published returns False for draft articles"""
        self.assertFalse(self.draft_article.is_published())
    
    def test_get_word_count(self):
        """Test word count calculation"""
        self.assertEqual(self.published_article.get_word_count(), 7)
        self.assertEqual(self.draft_article.get_word_count(), 5)
    
    def test_get_reading_time(self):
        """Test reading time calculation"""
        # 7 words / 200 wpm = 0.035 minutes, rounded up to 1
        self.assertEqual(self.published_article.get_reading_time(), 1)
```

### Testing Model Validation

#### Testing Field Validation
```python
from django.core.exceptions import ValidationError

class ArticleValidationTest(TestCase):
    def test_title_max_length(self):
        """Test that title field enforces max length"""
        long_title = "A" * 201  # Exceeds max_length=200
        article = Article(title=long_title, content="Test")
        
        with self.assertRaises(ValidationError):
            article.full_clean()
    
    def test_pub_date_not_in_future(self):
        """Test custom validation for publication date"""
        future_date = timezone.now() + timezone.timedelta(days=1)
        article = Article(
            title="Future Article",
            content="Test",
            pub_date=future_date,
            status="published"
        )
        
        with self.assertRaises(ValidationError):
            article.full_clean()
```

### Testing Model Managers

#### Testing Custom Managers
```python
# models.py
class PublishedManager(models.Manager):
    def get_queryset(self):
        return super().get_queryset().filter(
            status='published',
            pub_date__lte=timezone.now()
        )

class Article(models.Model):
    # ... fields ...
    objects = models.Manager()
    published = PublishedManager()

# tests/test_managers.py
class ArticleManagerTest(TestCase):
    def setUp(self):
        # Create test articles with different statuses
        self.published_article = Article.objects.create(
            title="Published",
            status="published",
            pub_date=timezone.now() - timezone.timedelta(days=1)
        )
        self.draft_article = Article.objects.create(
            title="Draft",
            status="draft"
        )
        self.future_article = Article.objects.create(
            title="Future",
            status="published",
            pub_date=timezone.now() + timezone.timedelta(days=1)
        )
    
    def test_published_manager_returns_only_published(self):
        """Test that published manager returns only published articles"""
        published_articles = Article.published.all()
        
        self.assertEqual(published_articles.count(), 1)
        self.assertIn(self.published_article, published_articles)
        self.assertNotIn(self.draft_article, published_articles)
        self.assertNotIn(self.future_article, published_articles)
    
    def test_default_manager_returns_all(self):
        """Test that default manager returns all articles"""
        all_articles = Article.objects.all()
        self.assertEqual(all_articles.count(), 3)
```

### Testing Utility Functions

#### Testing Helper Functions
```python
# utils.py
def slugify_title(title):
    """Convert title to URL-friendly slug"""
    import re
    from django.utils.text import slugify
    
    # Remove special characters and extra spaces
    title = re.sub(r'[^\w\s-]', '', title)
    return slugify(title)

def calculate_reading_time(content, words_per_minute=200):
    """Calculate estimated reading time in minutes"""
    word_count = len(content.split())
    minutes = word_count / words_per_minute
    return max(1, round(minutes))

# tests/test_utils.py
from myapp.utils import slugify_title, calculate_reading_time

class UtilsTest(TestCase):
    def test_slugify_title_basic(self):
        """Test basic slugification"""
        result = slugify_title("Hello World!")
        self.assertEqual(result, "hello-world")
    
    def test_slugify_title_special_chars(self):
        """Test slugification with special characters"""
        result = slugify_title("Hello @ World # Test!")
        self.assertEqual(result, "hello-world-test")
    
    def test_calculate_reading_time(self):
        """Test reading time calculation"""
        # 400 words at 200 wpm = 2 minutes
        content = "word " * 400
        result = calculate_reading_time(content)
        self.assertEqual(result, 2)
    
    def test_calculate_reading_time_minimum(self):
        """Test minimum reading time of 1 minute"""
        content = "short"
        result = calculate_reading_time(content)
        self.assertEqual(result, 1)
```

### Testing Signals

#### Testing Django Signals
```python
# models.py
from django.db.models.signals import post_save
from django.dispatch import receiver

@receiver(post_save, sender=Article)
def create_article_slug(sender, instance, created, **kwargs):
    if created and not instance.slug:
        instance.slug = slugify_title(instance.title)
        instance.save(update_fields=['slug'])

# tests/test_signals.py
from django.db.models.signals import post_save
from unittest.mock import patch

class ArticleSignalTest(TestCase):
    def test_slug_created_on_save(self):
        """Test that slug is automatically created when article is saved"""
        article = Article.objects.create(
            title="Test Article Title",
            content="Test content"
        )
        
        # Refresh from database
        article.refresh_from_db()
        self.assertEqual(article.slug, "test-article-title")
    
    @patch('myapp.models.slugify_title')
    def test_slugify_called_with_title(self, mock_slugify):
        """Test that slugify is called with the correct title"""
        mock_slugify.return_value = "mocked-slug"
        
        article = Article.objects.create(
            title="Test Title",
            content="Test content"
        )
        
        mock_slugify.assert_called_once_with("Test Title")
        
        article.refresh_from_db()
        self.assertEqual(article.slug, "mocked-slug")
```

## Integration Testing

Integration tests verify that different parts of your application work together correctly. This includes testing views, forms, and the interaction between components.

### Testing Views

#### Basic View Testing
```python
# tests/test_views.py
from django.test import TestCase, Client
from django.urls import reverse
from myapp.models import Article

class ArticleViewTest(TestCase):
    def setUp(self):
        self.client = Client()
        self.article = Article.objects.create(
            title="Test Article",
            content="Test content",
            status="published"
        )
    
    def test_article_list_view(self):
        """Test article list view returns correct response"""
        response = self.client.get(reverse('article_list'))
        
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Test Article")
        self.assertTemplateUsed(response, 'articles/article_list.html')
    
    def test_article_detail_view(self):
        """Test article detail view"""
        response = self.client.get(
            reverse('article_detail', kwargs={'pk': self.article.pk})
        )
        
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Test Article")
        self.assertContains(response, "Test content")
    
    def test_article_detail_view_not_found(self):
        """Test article detail view with invalid ID"""
        response = self.client.get(
            reverse('article_detail', kwargs={'pk': 999})
        )
        
        self.assertEqual(response.status_code, 404)
```

### Testing Forms

#### Form Testing
```python
# forms.py
from django import forms
from .models import Article

class ArticleForm(forms.ModelForm):
    class Meta:
        model = Article
        fields = ['title', 'content', 'status']
    
    def clean_title(self):
        title = self.cleaned_data['title']
        if len(title) < 5:
            raise forms.ValidationError("Title must be at least 5 characters long")
        return title

# tests/test_forms.py
from myapp.forms import ArticleForm

class ArticleFormTest(TestCase):
    def test_valid_form(self):
        """Test form with valid data"""
        form_data = {
            'title': 'Valid Title',
            'content': 'Valid content',
            'status': 'draft'
        }
        form = ArticleForm(data=form_data)
        
        self.assertTrue(form.is_valid())
        article = form.save()
        self.assertEqual(article.title, 'Valid Title')
    
    def test_invalid_title_too_short(self):
        """Test form validation for short title"""
        form_data = {
            'title': 'Hi',  # Too short
            'content': 'Valid content',
            'status': 'draft'
        }
        form = ArticleForm(data=form_data)
        
        self.assertFalse(form.is_valid())
        self.assertIn('title', form.errors)
        self.assertEqual(
            form.errors['title'],
            ['Title must be at least 5 characters long']
        )
    
    def test_form_save_creates_article(self):
        """Test that form.save() creates an article"""
        initial_count = Article.objects.count()
        
        form_data = {
            'title': 'New Article',
            'content': 'New content',
            'status': 'published'
        }
        form = ArticleForm(data=form_data)
        
        self.assertTrue(form.is_valid())
        article = form.save()
        
        self.assertEqual(Article.objects.count(), initial_count + 1)
        self.assertEqual(article.status, 'published')
```

### Testing Authentication and Permissions

#### Testing Login Required Views
```python
from django.contrib.auth.models import User

class AuthenticationTest(TestCase):
    def setUp(self):
        self.user = User.objects.create_user(
            username='testuser',
            password='testpass123'
        )
        self.client = Client()
    
    def test_login_required_redirect(self):
        """Test that login-required view redirects anonymous users"""
        response = self.client.get(reverse('article_create'))
        
        self.assertEqual(response.status_code, 302)
        self.assertIn('/login/', response['Location'])
    
    def test_authenticated_user_can_access(self):
        """Test that authenticated users can access protected views"""
        self.client.login(username='testuser', password='testpass123')
        response = self.client.get(reverse('article_create'))
        
        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, 'articles/article_form.html')
    
    def test_logout(self):
        """Test user logout"""
        self.client.login(username='testuser', password='testpass123')
        
        # Verify logged in
        response = self.client.get(reverse('article_create'))
        self.assertEqual(response.status_code, 200)
        
        # Logout
        self.client.logout()
        
        # Verify logged out
        response = self.client.get(reverse('article_create'))
        self.assertEqual(response.status_code, 302)
```

### Testing AJAX Views

#### Testing JSON Responses
```python
import json

class AjaxViewTest(TestCase):
    def test_ajax_article_like(self):
        """Test AJAX endpoint for liking articles"""
        self.client.login(username='testuser', password='testpass123')
        
        response = self.client.post(
            reverse('article_like', kwargs={'pk': self.article.pk}),
            HTTP_X_REQUESTED_WITH='XMLHttpRequest'  # Mark as AJAX
        )
        
        self.assertEqual(response.status_code, 200)
        
        # Parse JSON response
        data = json.loads(response.content)
        self.assertTrue(data['success'])
        self.assertEqual(data['likes_count'], 1)
    
    def test_ajax_requires_login(self):
        """Test that AJAX endpoints require authentication"""
        response = self.client.post(
            reverse('article_like', kwargs={'pk': self.article.pk}),
            HTTP_X_REQUESTED_WITH='XMLHttpRequest'
        )
        
        self.assertEqual(response.status_code, 403)
        
        data = json.loads(response.content)
        self.assertFalse(data['success'])
        self.assertIn('login required', data['message'].lower())
```

### Testing File Uploads

#### Testing File Upload Views
```python
from django.core.files.uploadedfile import SimpleUploadedFile
from PIL import Image
import io

class FileUploadTest(TestCase):
    def test_image_upload(self):
        """Test image upload functionality"""
        # Create a test image
        image = Image.new('RGB', (100, 100), color='red')
        image_io = io.BytesIO()
        image.save(image_io, format='JPEG')
        image_io.seek(0)
        
        uploaded_file = SimpleUploadedFile(
            "test_image.jpg",
            image_io.getvalue(),
            content_type="image/jpeg"
        )
        
        # Test the upload
        response = self.client.post(
            reverse('upload_image'),
            {'image': uploaded_file},
            format='multipart'
        )
        
        self.assertEqual(response.status_code, 200)
        
        # Check that image was saved
        self.assertTrue(
            ArticleImage.objects.filter(
                article=self.article,
                image__icontains='test_image'
            ).exists()
        )
    
    def test_invalid_file_type(self):
        """Test upload validation for invalid file types"""
        invalid_file = SimpleUploadedFile(
            "test.exe",
            b"fake executable content",
            content_type="application/octet-stream"
        )
        
        response = self.client.post(
            reverse('upload_image'),
            {'image': invalid_file}
        )
        
        self.assertEqual(response.status_code, 400)
        self.assertContains(response, "Invalid file type")
```

### Testing Email Sending

#### Testing Email Functionality
```python
from django.core import mail

class EmailTest(TestCase):
    def test_contact_form_sends_email(self):
        """Test that contact form sends email"""
        form_data = {
            'name': 'Test User',
            'email': 'test@example.com',
            'message': 'Test message'
        }
        
        response = self.client.post(
            reverse('contact'),
            form_data
        )
        
        self.assertEqual(response.status_code, 200)
        
        # Check that email was sent
        self.assertEqual(len(mail.outbox), 1)
        
        email = mail.outbox[0]
        self.assertEqual(email.subject, 'New Contact Form Submission')
        self.assertEqual(email.to, ['admin@example.com'])
        self.assertIn('Test User', email.body)
        self.assertIn('test@example.com', email.body)
        self.assertIn('Test message', email.body)
    
    def test_email_fields(self):
        """Test email content and formatting"""
        # Send test email
        mail.send_mail(
            'Test Subject',
            'Test body',
            'from@example.com',
            ['to@example.com']
        )
        
        # Check email details
        email = mail.outbox[0]
        self.assertEqual(email.subject, 'Test Subject')
        self.assertEqual(email.body, 'Test body')
        self.assertEqual(email.from_email, 'from@example.com')
        self.assertEqual(email.to, ['to@example.com'])
```

## API Testing

Testing REST APIs and GraphQL endpoints in Django applications.

### Testing Django REST Framework APIs

#### Basic API Testing
```python
# tests/test_api.py
from rest_framework.test import APITestCase
from rest_framework import status
from django.urls import reverse
from myapp.models import Article

class ArticleAPITest(APITestCase):
    def setUp(self):
        self.article = Article.objects.create(
            title="API Test Article",
            content="Test content",
            status="published"
        )
        self.user = User.objects.create_user(
            username='apiuser',
            password='testpass123'
        )
    
    def test_get_articles_list(self):
        """Test GET request to articles list endpoint"""
        url = reverse('article-list')
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data), 1)
        self.assertEqual(response.data[0]['title'], 'API Test Article')
    
    def test_get_article_detail(self):
        """Test GET request to article detail endpoint"""
        url = reverse('article-detail', kwargs={'pk': self.article.pk})
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(response.data['title'], 'API Test Article')
        self.assertEqual(response.data['content'], 'Test content')
    
    def test_create_article_authenticated(self):
        """Test POST request to create article (authenticated)"""
        self.client.force_authenticate(user=self.user)
        
        url = reverse('article-list')
        data = {
            'title': 'New Article',
            'content': 'New content',
            'status': 'draft'
        }
        response = self.client.post(url, data, format='json')
        
        self.assertEqual(response.status_code, status.HTTP_201_CREATED)
        self.assertEqual(Article.objects.count(), 2)
        self.assertEqual(response.data['title'], 'New Article')
    
    def test_create_article_unauthenticated(self):
        """Test POST request to create article (unauthenticated)"""
        url = reverse('article-list')
        data = {
            'title': 'New Article',
            'content': 'New content'
        }
        response = self.client.post(url, data, format='json')
        
        self.assertEqual(response.status_code, status.HTTP_401_UNAUTHORIZED)
```

### Testing API Permissions

#### Permission Testing
```python
from rest_framework.test import APIClient

class ArticlePermissionsTest(APITestCase):
    def setUp(self):
        self.client = APIClient()
        
        # Create users with different permissions
        self.author = User.objects.create_user(
            username='author',
            password='pass'
        )
        self.editor = User.objects.create_user(
            username='editor',
            password='pass'
        )
        
        # Create article
        self.article = Article.objects.create(
            title="Permission Test",
            content="Test content",
            author=self.author
        )
    
    def test_author_can_edit_own_article(self):
        """Test that authors can edit their own articles"""
        self.client.force_authenticate(user=self.author)
        
        url = reverse('article-detail', kwargs={'pk': self.article.pk})
        data = {'title': 'Updated Title'}
        response = self.client.patch(url, data, format='json')
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.article.refresh_from_db()
        self.assertEqual(self.article.title, 'Updated Title')
    
    def test_editor_cannot_edit_others_article(self):
        """Test that editors cannot edit others' articles"""
        self.client.force_authenticate(user=self.editor)
        
        url = reverse('article-detail', kwargs={'pk': self.article.pk})
        data = {'title': 'Hacked Title'}
        response = self.client.patch(url, data, format='json')
        
        self.assertEqual(response.status_code, status.HTTP_403_FORBIDDEN)
```

### Testing API Serializers

#### Serializer Testing
```python
# tests/test_serializers.py
from myapp.serializers import ArticleSerializer

class ArticleSerializerTest(TestCase):
    def setUp(self):
        self.article = Article.objects.create(
            title="Serializer Test",
            content="Test content",
            status="published"
        )
    
    def test_serialize_article(self):
        """Test serializing an article"""
        serializer = ArticleSerializer(self.article)
        
        expected_data = {
            'id': self.article.id,
            'title': 'Serializer Test',
            'content': 'Test content',
            'status': 'published',
            'pub_date': self.article.pub_date.isoformat()[:-6] + 'Z',
        }
        
        self.assertEqual(serializer.data, expected_data)
    
    def test_deserialize_valid_data(self):
        """Test deserializing valid data"""
        data = {
            'title': 'New Article',
            'content': 'New content',
            'status': 'draft'
        }
        serializer = ArticleSerializer(data=data)
        
        self.assertTrue(serializer.is_valid())
        article = serializer.save()
        
        self.assertEqual(article.title, 'New Article')
        self.assertEqual(article.status, 'draft')
    
    def test_deserialize_invalid_data(self):
        """Test deserializing invalid data"""
        data = {
            'title': '',  # Invalid: empty title
            'content': 'Valid content',
            'status': 'invalid_status'  # Invalid: not a choice
        }
        serializer = ArticleSerializer(data=data)
        
        self.assertFalse(serializer.is_valid())
        self.assertIn('title', serializer.errors)
        self.assertIn('status', serializer.errors)
```

### Testing Pagination

#### API Pagination Testing
```python
class ArticlePaginationTest(APITestCase):
    def setUp(self):
        # Create multiple articles
        for i in range(25):
            Article.objects.create(
                title=f"Article {i}",
                content=f"Content {i}",
                status="published"
            )
    
    def test_pagination_first_page(self):
        """Test first page of paginated results"""
        url = reverse('article-list')
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data['results']), 10)  # Default page size
        self.assertIsNotNone(response.data['next'])
        self.assertIsNone(response.data['previous'])
    
    def test_pagination_second_page(self):
        """Test second page of paginated results"""
        url = reverse('article-list') + '?page=2'
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data['results']), 10)
        self.assertIsNotNone(response.data['next'])
        self.assertIsNotNone(response.data['previous'])
    
    def test_pagination_last_page(self):
        """Test last page of paginated results"""
        url = reverse('article-list') + '?page=3'
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data['results']), 5)  # Remaining items
        self.assertIsNone(response.data['next'])
        self.assertIsNotNone(response.data['previous'])
```

### Testing API Filtering and Search

#### Filter and Search Testing
```python
class ArticleFilteringTest(APITestCase):
    def setUp(self):
        # Create articles with different statuses and content
        Article.objects.create(title="Python Guide", content="Learn Python", status="published")
        Article.objects.create(title="Django Tutorial", content="Learn Django", status="published")
        Article.objects.create(title="Draft Article", content="Draft content", status="draft")
    
    def test_filter_by_status(self):
        """Test filtering articles by status"""
        url = reverse('article-list') + '?status=published'
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data), 2)
        
        for article in response.data:
            self.assertEqual(article['status'], 'published')
    
    def test_search_functionality(self):
        """Test search functionality"""
        url = reverse('article-list') + '?search=Django'
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertEqual(len(response.data), 1)
        self.assertEqual(response.data[0]['title'], 'Django Tutorial')
    
    def test_ordering(self):
        """Test ordering functionality"""
        url = reverse('article-list') + '?ordering=-pub_date'
        response = self.client.get(url)
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        # Verify articles are ordered by publication date (newest first)
        dates = [article['pub_date'] for article in response.data]
        self.assertEqual(dates, sorted(dates, reverse=True))
```

### Testing GraphQL APIs

#### GraphQL Testing
```python
# tests/test_graphql.py
from graphene.test import Client
from myapp.schema import schema

class GraphQLTest(TestCase):
    def setUp(self):
        self.client = Client(schema)
        self.article = Article.objects.create(
            title="GraphQL Test",
            content="Test content",
            status="published"
        )
    
    def test_get_articles_query(self):
        """Test GraphQL query for articles"""
        query = '''
        query {
            articles {
                id
                title
                content
            }
        }
        '''
        
        result = self.client.execute(query)
        
        self.assertIsNone(result.get('errors'))
        self.assertEqual(len(result['data']['articles']), 1)
        self.assertEqual(
            result['data']['articles'][0]['title'],
            'GraphQL Test'
        )
    
    def test_create_article_mutation(self):
        """Test GraphQL mutation for creating articles"""
        mutation = '''
        mutation {
            createArticle(
                title: "New Article"
                content: "New content"
                status: PUBLISHED
            ) {
                article {
                    id
                    title
                    status
                }
                success
            }
        }
        '''
        
        result = self.client.execute(mutation)
        
        self.assertIsNone(result.get('errors'))
        self.assertTrue(result['data']['createArticle']['success'])
        self.assertEqual(
            result['data']['createArticle']['article']['title'],
            'New Article'
        )
```

### Testing API Rate Limiting

#### Rate Limit Testing
```python
from django.core.cache import cache

class APIRateLimitTest(APITestCase):
    def setUp(self):
        self.user = User.objects.create_user(
            username='testuser',
            password='testpass'
        )
        self.client.force_authenticate(user=self.user)
    
    def test_rate_limit_not_exceeded(self):
        """Test API calls within rate limit"""
        url = reverse('article-list')
        
        # Make requests within limit (assuming 10 per minute)
        for i in range(5):
            response = self.client.get(url)
            self.assertEqual(response.status_code, status.HTTP_200_OK)
    
    def test_rate_limit_exceeded(self):
        """Test API calls exceeding rate limit"""
        url = reverse('article-list')
        
        # Clear any existing rate limit data
        cache.clear()
        
        # Make requests that exceed limit
        for i in range(15):  # Assuming limit is 10
            response = self.client.get(url)
            
            if i < 10:
                self.assertEqual(response.status_code, status.HTTP_200_OK)
            else:
                self.assertEqual(response.status_code, status.HTTP_429_TOO_MANY_REQUESTS)
```

## Test Database Management

Understanding how Django manages test databases and optimizing test performance.

### Test Database Creation

#### Default Test Database Behavior
```python
# Django automatically creates a test database
# Database name: test_<original_db_name>
# All migrations are applied automatically

# settings.py for test database configuration
if 'test' in sys.argv:
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': ':memory:',  # Use in-memory database for faster tests
        }
    }
```

### Fixtures

#### Creating and Using Fixtures
```python
# Create fixture from existing data
# python manage.py dumpdata myapp.Article --indent=2 > articles.json

# tests/test_with_fixtures.py
class ArticleFixtureTest(TestCase):
    fixtures = ['articles.json', 'iam.json']
    
    def test_articles_loaded_from_fixture(self):
        """Test that fixtures are loaded correctly"""
        articles = Article.objects.all()
        self.assertGreater(articles.count(), 0)
        
        # Test specific data from fixture
        featured_article = Article.objects.get(title="Featured Article")
        self.assertEqual(featured_article.status, 'published')
```

#### Fixture Loading
```python
# Manual fixture loading
from django.core.management import call_command
from django.test.utils import override_settings

class ManualFixtureTest(TestCase):
    def setUp(self):
        # Load fixtures manually
        call_command('loaddata', 'test_data.json', verbosity=0)
    
    def test_with_manual_fixture_loading(self):
        """Test with manually loaded fixtures"""
        # Test logic here
        pass

# Conditional fixture loading
class ConditionalFixtureTest(TestCase):
    @classmethod
    def setUpTestData(cls):
        # Create test data once for all test methods
        if not Article.objects.exists():
            # Create initial data
            pass
```

### Test Data Factories

#### Using Factory Boy
```python
# factories.py
import factory
from myapp.models import Article, Author

class AuthorFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Author
    
    name = factory.Faker('name')
    email = factory.Faker('email')
    bio = factory.Faker('text', max_nb_chars=200)

class ArticleFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Article
    
    title = factory.Faker('sentence', nb_words=4)
    content = factory.Faker('text', max_nb_chars=1000)
    status = 'draft'
    author = factory.SubFactory(AuthorFactory)
    pub_date = factory.Faker('date_time_this_year')

# tests/test_factories.py
from .factories import ArticleFactory, AuthorFactory

class FactoryTest(TestCase):
    def test_create_article_with_factory(self):
        """Test creating article using factory"""
        article = ArticleFactory()
        
        self.assertIsNotNone(article.title)
        self.assertIsNotNone(article.author)
        self.assertEqual(article.status, 'draft')
    
    def test_create_multiple_articles(self):
        """Test creating multiple articles with relationships"""
        articles = ArticleFactory.create_batch(5)
        
        self.assertEqual(len(articles), 5)
        
        # All articles should have different authors
        author_ids = [article.author.id for article in articles]
        self.assertEqual(len(set(author_ids)), 5)
    
    def test_factory_with_custom_attributes(self):
        """Test factory with custom attribute overrides"""
        article = ArticleFactory(
            title="Custom Title",
            status="published"
        )
        
        self.assertEqual(article.title, "Custom Title")
        self.assertEqual(article.status, "published")
```

### Transaction Management

#### TransactionTestCase vs TestCase
```python
# TestCase: Faster, uses transactions
class FastTest(TestCase):
    def test_something(self):
        # Changes are rolled back automatically
        Article.objects.create(title="Test")
        self.assertEqual(Article.objects.count(), 1)
    
    # After test, database is rolled back

# TransactionTestCase: Slower, truncates tables
class DatabaseOperationTest(TransactionTestCase):
    def test_database_operations(self):
        # Useful for testing database triggers, constraints
        # or when you need to test across transactions
        pass
```

#### Manual Transaction Control
```python
from django.db import transaction
from django.test import TransactionTestCase

class TransactionControlTest(TransactionTestCase):
    def test_transaction_rollback(self):
        """Test manual transaction control"""
        try:
            with transaction.atomic():
                Article.objects.create(title="Should not persist")
                raise Exception("Force rollback")
        except Exception:
            pass
        
        # Article should not exist due to rollback
        self.assertEqual(Article.objects.count(), 0)
```

### Test Performance Optimization

#### setUpTestData for Class-Level Setup
```python
class OptimizedTest(TestCase):
    @classmethod
    def setUpTestData(cls):
        """Set up data once for entire test class"""
        cls.author = Author.objects.create(name="Test Author")
        cls.articles = Article.objects.bulk_create([
            Article(title=f"Article {i}", author=cls.author)
            for i in range(10)
        ])
    
    def test_article_count(self):
        """Test runs faster with pre-created data"""
        self.assertEqual(len(self.articles), 10)
    
    def test_article_titles(self):
        """Another test using the same data"""
        titles = [article.title for article in self.articles]
        self.assertIn("Article 5", titles)
```

#### Database Query Optimization in Tests
```python
class QueryOptimizationTest(TestCase):
    def test_no_unnecessary_queries(self):
        """Test that view doesn't make unnecessary queries"""
        # Create test data
        author = Author.objects.create(name="Test")
        articles = [
            Article.objects.create(title=f"Article {i}", author=author)
            for i in range(5)
        ]
        
        # Test the view
        with self.assertNumQueries(2):  # Adjust expected query count
            response = self.client.get(reverse('article_list'))
            self.assertEqual(response.status_code, 200)
    
    def test_select_related_usage(self):
        """Test that select_related is used properly"""
        author = Author.objects.create(name="Test")
        article = Article.objects.create(title="Test", author=author)
        
        # Should only make 1 query with select_related
        with self.assertNumQueries(1):
            article_with_author = Article.objects.select_related('author').get(pk=article.pk)
            _ = article_with_author.author.name  # Access related object
```

### Test Parallelization

#### Running Tests in Parallel
```bash
# Run tests in parallel (Django 2.2+)
python manage.py test --parallel auto

# Specify number of processes
python manage.py test --parallel 4

# Run specific test classes in parallel
python manage.py test myapp.tests.ArticleTest myapp.tests.AuthorTest --parallel 2
```

#### Parallel Test Considerations
```python
# Tests that modify global state need special handling
class ParallelSafeTest(TestCase):
    # Avoid tests that:
    # - Modify settings
    # - Use global caches
    # - Depend on external services without proper isolation
    
    def test_isolated_functionality(self):
        """Test that can run safely in parallel"""
        # Test logic that doesn't interfere with other tests
        pass

# For tests that need isolation
class IsolatedTest(TransactionTestCase):
    """Use TransactionTestCase for tests that need full isolation"""
    
    def test_database_isolation(self):
        """Test that runs with full database isolation"""
        # Useful for testing database constraints, triggers, etc.
        pass
```

### Test Database Configuration

#### Custom Test Database Settings
```python
# settings.py
import sys

if 'test' in sys.argv:
    # Use faster database for tests
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': ':memory:',
        }
    }
    
    # Disable some middleware in tests for speed
    MIDDLEWARE = [
        mw for mw in MIDDLEWARE 
        if not mw.endswith('SecurityMiddleware')
    ]
    
    # Use faster password hasher
    PASSWORD_HASHERS = [
        'django.contrib.auth.hashers.MD5PasswordHasher',
    ]
    
    # Disable email sending
    EMAIL_BACKEND = 'django.core.mail.backends.locmem.EmailBackend'
```

#### Test-Specific Settings
```python
# test_settings.py
from .settings import *

# Override settings for testing
SECRET_KEY = 'test-secret-key'
DEBUG = False
USE_TZ = False  # Faster tests without timezone handling

# Use test-specific media root
MEDIA_ROOT = '/tmp/test_media'
STATIC_ROOT = '/tmp/test_static'

# Configure test cache
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
    }
}
```

### Database State Management

#### Resetting Database State
```python
class DatabaseResetTest(TestCase):
    def setUp(self):
        # Ensure clean state
        Article.objects.all().delete()
        Author.objects.all().delete()
    
    def test_database_reset(self):
        """Test that database is properly reset between tests"""
        # Create some data
        Article.objects.create(title="Test Article")
        self.assertEqual(Article.objects.count(), 1)
        
        # Next test should start with clean database
        # (This is automatic with Django's TestCase)

# For tests that need specific database state
class SpecificStateTest(TestCase):
    def setUp(self):
        # Set up specific state for this test
        self.article = Article.objects.create(
            title="Specific Test Article",
            status="published"
        )
    
    def test_specific_state(self):
        """Test that depends on specific database state"""
        self.assertEqual(self.article.status, "published")
```

### Test Cleanup

#### Automatic vs Manual Cleanup
```python
class CleanupTest(TestCase):
    def setUp(self):
        # Create test files
        self.test_file = '/tmp/test_file.txt'
        with open(self.test_file, 'w') as f:
            f.write('test content')
    
    def tearDown(self):
        # Clean up test files
        if os.path.exists(self.test_file):
            os.remove(self.test_file)
    
    def test_file_operations(self):
        """Test that creates and cleans up files"""
        self.assertTrue(os.path.exists(self.test_file))
        
        # File should be cleaned up in tearDown

# Using context managers for cleanup
class ContextManagerTest(TestCase):
    def test_with_context_manager(self):
        """Test using context managers for automatic cleanup"""
        with tempfile.NamedTemporaryFile() as temp_file:
            temp_file.write(b'test content')
            temp_file.flush()
            
            # Test file operations
            self.assertTrue(os.path.exists(temp_file.name))
        
        # File is automatically cleaned up
```

## Mocking and Test Fixtures

Using mocks and fixtures to isolate tests and control test data.

### Using unittest.mock

#### Basic Mocking
```python
# tests/test_mocking.py
from unittest.mock import Mock, patch, MagicMock
from myapp.services import ExternalAPIService

class MockingTest(TestCase):
    def test_external_api_call(self):
        """Test external API call with mocking"""
        # Create a mock response
        mock_response = {
            'status': 'success',
            'data': {'key': 'value'}
        }
        
        # Mock the external API call
        with patch('myapp.services.requests.get') as mock_get:
            mock_get.return_value.json.return_value = mock_response
            
            service = ExternalAPIService()
            result = service.get_data()
            
            self.assertEqual(result['status'], 'success')
            mock_get.assert_called_once()
    
    def test_method_with_side_effects(self):
        """Test method with side effects using mock"""
        mock_obj = Mock()
        mock_obj.some_method.return_value = 'mocked result'
        mock_obj.some_method.side_effect = print  # Print when called
        
        result = mock_obj.some_method('arg')
        
        self.assertEqual(result, 'mocked result')
        mock_obj.some_method.assert_called_with('arg')
```

### Mocking Django Components

#### Mocking Models
```python
from unittest.mock import patch

class ModelMockingTest(TestCase):
    @patch('myapp.models.Article.objects')
    def test_model_method_with_mock(self, mock_objects):
        """Test model method with mocked queryset"""
        # Configure the mock
        mock_queryset = Mock()
        mock_queryset.filter.return_value = mock_queryset
        mock_queryset.count.return_value = 5
        mock_objects.filter.return_value = mock_queryset
        
        from myapp.views import article_count_view
        result = article_count_view()
        
        self.assertEqual(result, 5)
        mock_objects.filter.assert_called_with(status='published')
    
    def test_model_save_with_mock(self):
        """Test model save with mocking"""
        with patch('myapp.models.Article.save') as mock_save:
            article = Article(title="Test")
            article.save()
            
            mock_save.assert_called_once()
```

#### Mocking Email Sending
```python
from django.core import mail

class EmailMockingTest(TestCase):
    def setUp(self):
        # Clear the email outbox before each test
        mail.outbox = []
    
    def test_email_sending(self):
        """Test email sending functionality"""
        from django.core.mail import send_mail
        
        send_mail(
            'Test Subject',
            'Test message',
            'from@example.com',
            ['to@example.com']
        )
        
        # Check that email was sent
        self.assertEqual(len(mail.outbox), 1)
        email = mail.outbox[0]
        self.assertEqual(email.subject, 'Test Subject')
        self.assertEqual(email.to, ['to@example.com'])
    
    @patch('myapp.tasks.send_welcome_email.delay')
    def test_async_email_task(self, mock_delay):
        """Test async email task with mocking"""
        from myapp.views import register_user
        
        # Call the view
        response = self.client.post('/register/', {
            'username': 'testuser',
            'email': 'test@example.com'
        })
        
        # Check that the async task was called
        mock_delay.assert_called_once_with('test@example.com')
```

### Mocking External Services

#### Mocking HTTP Requests
```python
import responses
from myapp.services import WeatherService

class ExternalServiceTest(TestCase):
    @responses.activate
    def test_weather_api_call(self):
        """Test external API call with responses library"""
        # Mock the API response
        responses.add(
            responses.GET,
            'https://api.weather.com/current',
            json={'temperature': 22, 'condition': 'sunny'},
            status=200
        )
        
        service = WeatherService()
        weather = service.get_current_weather('New York')
        
        self.assertEqual(weather['temperature'], 22)
        self.assertEqual(weather['condition'], 'sunny')
    
    @responses.activate
    def test_api_error_handling(self):
        """Test API error handling"""
        responses.add(
            responses.GET,
            'https://api.weather.com/current',
            status=500
        )
        
        service = WeatherService()
        with self.assertRaises(Exception):
            service.get_current_weather('New York')
```

### Creating Test Fixtures

#### JSON Fixtures
```json
// fixtures/test_data.json
[
  {
    "model": "myapp.author",
    "pk": 1,
    "fields": {
      "name": "Test Author",
      "email": "author@example.com",
      "bio": "Test biography"
    }
  },
  {
    "model": "myapp.article",
    "pk": 1,
    "fields": {
      "title": "Test Article",
      "content": "Test content",
      "status": "published",
      "author": 1,
      "pub_date": "2024-01-01T12:00:00Z"
    }
  }
]
```

#### Using Fixtures in Tests
```python
class FixtureTest(TestCase):
    fixtures = ['test_data.json']
    
    def test_fixture_data_loaded(self):
        """Test that fixture data is loaded correctly"""
        author = Author.objects.get(pk=1)
        self.assertEqual(author.name, 'Test Author')
        
        article = Article.objects.get(pk=1)
        self.assertEqual(article.title, 'Test Article')
        self.assertEqual(article.author, author)
    
    def test_relationships_preserved(self):
        """Test that relationships are preserved in fixtures"""
        article = Article.objects.get(pk=1)
        self.assertEqual(article.author.name, 'Test Author')
```

### Advanced Mocking Patterns

#### Mocking Class Methods
```python
class AdvancedMockingTest(TestCase):
    @patch.object(Article, 'get_reading_time')
    def test_class_method_mocking(self, mock_method):
        """Test mocking class methods"""
        mock_method.return_value = 5
        
        article = Article.objects.create(title="Test")
        result = article.get_reading_time()
        
        self.assertEqual(result, 5)
        mock_method.assert_called_once()
    
    @patch('myapp.utils.timezone.now')
    def test_time_dependent_function(self, mock_now):
        """Test function that depends on current time"""
        fixed_time = timezone.datetime(2024, 1, 1, 12, 0, 0, tzinfo=timezone.utc)
        mock_now.return_value = fixed_time
        
        from myapp.utils import get_current_time_string
        result = get_current_time_string()
        
        self.assertEqual(result, '2024-01-01 12:00:00')
```

#### Context Managers for Mocking
```python
from unittest.mock import patch
from contextlib import contextmanager

@contextmanager
def mock_external_service():
    """Context manager for mocking external service"""
    with patch('myapp.services.ExternalService.make_request') as mock_request:
        mock_request.return_value = {'status': 'success', 'data': 'mocked'}
        yield mock_request

class ContextManagerMockTest(TestCase):
    def test_with_context_manager_mock(self):
        """Test using context manager for mocking"""
        with mock_external_service() as mock_request:
            from myapp.services import ExternalService
            service = ExternalService()
            result = service.make_request('test')
            
            self.assertEqual(result['status'], 'success')
            mock_request.assert_called_with('test')
```

### Mocking Database Operations

#### Mocking Querysets
```python
class DatabaseMockingTest(TestCase):
    def test_queryset_mocking(self):
        """Test mocking database querysets"""
        mock_queryset = Mock()
        mock_queryset.filter.return_value = mock_queryset
        mock_queryset.order_by.return_value = mock_queryset
        mock_queryset.all.return_value = [
            Mock(title='Article 1'),
            Mock(title='Article 2')
        ]
        
        with patch('myapp.models.Article.objects', mock_queryset):
            from myapp.views import get_published_articles
            articles = get_published_articles()
            
            self.assertEqual(len(articles), 2)
            mock_queryset.filter.assert_called_with(status='published')
            mock_queryset.order_by.assert_called_with('-pub_date')
```

#### Mocking Model Instances
```python
class ModelInstanceMockTest(TestCase):
    def test_model_instance_mocking(self):
        """Test mocking model instances"""
        mock_article = Mock(spec=Article)
        mock_article.title = 'Mocked Title'
        mock_article.get_reading_time.return_value = 3
        mock_article.save.return_value = None
        
        # Test the mocked instance
        self.assertEqual(mock_article.title, 'Mocked Title')
        self.assertEqual(mock_article.get_reading_time(), 3)
        
        # Call save and verify it was called
        mock_article.save()
        mock_article.save.assert_called_once()
```

### Mocking File Operations

#### Mocking File Uploads
```python
from django.core.files.uploadedfile import SimpleUploadedFile

class FileMockingTest(TestCase):
    def test_file_upload_mocking(self):
        """Test file upload with mocking"""
        # Create a mock uploaded file
        mock_file = SimpleUploadedFile(
            "test.txt",
            b"file content",
            content_type="text/plain"
        )
        
        # Mock the file processing
        with patch('myapp.utils.process_uploaded_file') as mock_process:
            mock_process.return_value = 'processed'
            
            from myapp.views import upload_view
            response = self.client.post('/upload/', {'file': mock_file})
            
            self.assertEqual(response.status_code, 200)
            mock_process.assert_called_once()
    
    def test_file_system_operations(self):
        """Test file system operations with mocking"""
        with patch('os.path.exists') as mock_exists, \
             patch('builtins.open', create=True) as mock_open:
            
            mock_exists.return_value = True
            mock_file = Mock()
            mock_file.read.return_value = 'file content'
            mock_open.return_value.__enter__.return_value = mock_file
            
            from myapp.utils import read_file
            content = read_file('/path/to/file.txt')
            
            self.assertEqual(content, 'file content')
            mock_exists.assert_called_with('/path/to/file.txt')
```

### Mocking Time and Dates

#### Mocking Time-Dependent Code
```python
from unittest.mock import patch
from django.utils import timezone

class TimeMockingTest(TestCase):
    @patch('django.utils.timezone.now')
    def test_time_dependent_logic(self, mock_now):
        """Test logic that depends on current time"""
        # Set a fixed time
        fixed_time = timezone.datetime(2024, 1, 15, 10, 30, tzinfo=timezone.utc)
        mock_now.return_value = fixed_time
        
        from myapp.utils import is_business_hours
        result = is_business_hours()
        
        # 10:30 should be business hours (9-17)
        self.assertTrue(result)
    
    @patch('django.utils.timezone.now')
    def test_time_based_expiration(self, mock_now):
        """Test time-based expiration logic"""
        # Set time to after expiration
        expired_time = timezone.datetime(2024, 1, 16, 10, 0, tzinfo=timezone.utc)
        mock_now.return_value = expired_time
        
        from myapp.models import Session
        session = Session.objects.create(
            expires_at=timezone.datetime(2024, 1, 15, 10, 0, tzinfo=timezone.utc)
        )
        
        self.assertTrue(session.is_expired())
```

### Mocking Settings and Configuration

#### Mocking Django Settings
```python
from django.test import override_settings

class SettingsMockingTest(TestCase):
    @override_settings(DEBUG=True)
    def test_debug_enabled(self):
        """Test behavior when DEBUG is enabled"""
        from django.conf import settings
        self.assertTrue(settings.DEBUG)
        
        # Test debug-specific behavior
        from myapp.utils import get_debug_info
        info = get_debug_info()
        self.assertIn('debug', info)
    
    @override_settings(
        EMAIL_BACKEND='django.core.mail.backends.locmem.EmailBackend',
        ADMINS=[('Admin', 'admin@example.com')]
    )
    def test_email_configuration(self):
        """Test email configuration mocking"""
        from django.core.mail import send_mail
        from django.conf import settings
        
        send_mail('Test', 'Body', 'from@test.com', ['to@test.com'])
        
        # Check that email was stored in memory
        from django.core import mail
        self.assertEqual(len(mail.outbox), 1)
        self.assertEqual(settings.ADMINS[0][1], 'admin@example.com')
```

### Best Practices for Mocking

#### When to Use Mocks
- External API calls
- Database operations (in unit tests)
- File system operations
- Time-dependent code
- Expensive computations
- Third-party services

#### Mocking Anti-Patterns
```python
# Bad: Over-mocking (don't mock everything)
class OverMockedTest(TestCase):
    @patch('myapp.models.Article.objects')
    @patch('myapp.models.Author.objects')
    @patch('django.utils.timezone.now')
    @patch('myapp.utils.send_email')
    def test_over_mocked(self, mock_email, mock_now, mock_author, mock_article):
        # This test doesn't test real functionality
        pass

# Good: Mock only external dependencies
class ProperlyMockedTest(TestCase):
    @patch('myapp.services.external_api_call')
    def test_external_dependency_mocked(self, mock_api):
        mock_api.return_value = {'status': 'success'}
        
        # Test real business logic, mock external calls
        result = my_function_that_calls_external_api()
        self.assertEqual(result['status'], 'success')
```

#### Mock Verification
```python
class MockVerificationTest(TestCase):
    def test_mock_called_correctly(self):
        """Test that mocks are called with correct arguments"""
        with patch('myapp.services.send_notification') as mock_send:
            mock_send.return_value = True
            
            # Call the function being tested
            result = send_user_notification(user_id=1, message='Hello')
            
            # Verify the mock was called
            mock_send.assert_called_once_with(user_id=1, message='Hello')
            
            # Verify return value
            self.assertTrue(result)
    
    def test_mock_call_count(self):
        """Test that mock is called expected number of times"""
        with patch('myapp.tasks.process_item.delay') as mock_delay:
            # Process multiple items
            process_items_batch([1, 2, 3])
            
            # Should be called once for each item
            self.assertEqual(mock_delay.call_count, 3)
            
            # Verify call arguments
            mock_delay.assert_has_calls([
                ((1,), {}),
                ((2,), {}),
                ((3,), {})
            ])
```

## Test Organization and Structure

Best practices for organizing and structuring Django tests.

### Test File Organization

#### Standard Django Test Structure
```
myapp/
    tests/
        __init__.py
        test_models.py       # Model tests
        test_views.py        # View tests
        test_forms.py        # Form tests
        test_api.py          # API tests
        test_utils.py        # Utility function tests
        test_tasks.py        # Background task tests
        test_commands.py     # Management command tests
        test_middleware.py   # Middleware tests
        test_signals.py      # Signal tests
        fixtures/           # Test fixtures
            iam.json
            articles.json
        factories/          # Test factories
            __init__.py
            user_factory.py
            article_factory.py
```

#### Test Naming Conventions
```python
# Good test naming
class ArticleModelTest(TestCase):
    def test_creation_with_valid_data(self):
        """Test that article can be created with valid data"""
        pass
    
    def test_creation_fails_with_invalid_title(self):
        """Test that article creation fails with invalid title"""
        pass
    
    def test_publish_method_changes_status(self):
        """Test that publish method changes article status"""
        pass

# Bad test naming
class TestArticle:
    def test1(self):
        pass
    
    def test_article_creation(self):
        pass
```

### Test Base Classes

#### Creating Test Base Classes
```python
# tests/base.py
from django.test import TestCase
from django.contrib.auth.models import User

class BaseTestCase(TestCase):
    """Base test case with common setup"""
    
    def setUp(self):
        self.user = User.objects.create_user(
            username='testuser',
            password='testpass123'
        )
        self.client.login(username='testuser', password='testpass123')

class ArticleTestMixin:
    """Mixin for article-related test setup"""
    
    def create_test_article(self, **kwargs):
        """Helper method to create test articles"""
        defaults = {
            'title': 'Test Article',
            'content': 'Test content',
            'status': 'draft'
        }
        defaults.update(kwargs)
        return Article.objects.create(**defaults)

# Using base classes and mixins
class ArticleViewTest(BaseTestCase, ArticleTestMixin):
    def test_article_list_view(self):
        """Test article list view with authenticated user"""
        self.create_test_article(status='published')
        response = self.client.get(reverse('article_list'))
        self.assertEqual(response.status_code, 200)
```

### Test Categories

#### Unit Tests
```python
# tests/test_unit.py
from django.test import TestCase
from myapp.utils import calculate_discount

class UnitTestExample(TestCase):
    """Pure unit tests that test functions in isolation"""
    
    def test_calculate_discount_percentage(self):
        """Test discount calculation with percentage"""
        result = calculate_discount(100, '10%')
        self.assertEqual(result, 10)
    
    def test_calculate_discount_fixed(self):
        """Test discount calculation with fixed amount"""
        result = calculate_discount(100, '15')
        self.assertEqual(result, 15)
```

#### Integration Tests
```python
# tests/test_integration.py
from django.test import TestCase
from django.core import mail

class IntegrationTestExample(TestCase):
    """Tests that verify component interactions"""
    
    def test_user_registration_flow(self):
        """Test complete user registration flow"""
        # Test form submission
        response = self.client.post(reverse('register'), {
            'username': 'newuser',
            'email': 'new@example.com',
            'password1': 'password123',
            'password2': 'password123'
        })
        
        # Check redirect
        self.assertRedirects(response, reverse('profile'))
        
        # Check user was created
        user = User.objects.get(username='newuser')
        self.assertEqual(user.email, 'new@example.com')
        
        # Check welcome email was sent
        self.assertEqual(len(mail.outbox), 1)
        self.assertIn('Welcome', mail.outbox[0].subject)
```

#### End-to-End Tests
```python
# tests/test_e2e.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.by import By

class E2ETestExample(LiveServerTestCase):
    """End-to-end tests using Selenium"""
    
    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        cls.selenium = webdriver.Chrome()
        cls.selenium.implicitly_wait(10)
    
    @classmethod
    def tearDownClass(cls):
        cls.selenium.quit()
        super().tearDownClass()
    
    def test_user_registration_e2e(self):
        """Test user registration through browser"""
        self.selenium.get(f'{self.live_server_url}/register/')
        
        # Fill form
        username_input = self.selenium.find_element(By.NAME, 'username')
        username_input.send_keys('e2euser')
        
        email_input = self.selenium.find_element(By.NAME, 'email')
        email_input.send_keys('e2e@example.com')
        
        # Submit form
        submit_button = self.selenium.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        submit_button.click()
        
        # Check redirect to success page
        self.assertIn('/profile/', self.selenium.current_url)
```

### Test Data Management

#### Factory Pattern for Test Data
```python
# tests/factories.py
import factory
from django.contrib.auth.models import User
from myapp.models import Article, Author

class UserFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = User
    
    username = factory.Sequence(lambda n: f'user{n}')
    email = factory.LazyAttribute(lambda obj: f'{obj.username}@example.com')
    is_active = True

class AuthorFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Author
    
    user = factory.SubFactory(UserFactory)
    name = factory.Faker('name')
    bio = factory.Faker('text', max_nb_chars=200)

class ArticleFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Article
    
    title = factory.Faker('sentence', nb_words=4)
    content = factory.Faker('text', max_nb_chars=1000)
    author = factory.SubFactory(AuthorFactory)
    status = 'draft'
    pub_date = factory.Faker('date_time_this_year')

# Using factories in tests
class FactoryTestExample(TestCase):
    def test_article_creation_with_factory(self):
        """Test using factory for test data creation"""
        article = ArticleFactory(title="Custom Title")
        
        self.assertEqual(article.title, "Custom Title")
        self.assertIsNotNone(article.author)
        self.assertEqual(article.status, 'draft')
    
    def test_bulk_factory_creation(self):
        """Test creating multiple objects with factory"""
        articles = ArticleFactory.create_batch(5, status='published')
        
        self.assertEqual(len(articles), 5)
        for article in articles:
            self.assertEqual(article.status, 'published')
```

### Test Helpers and Utilities

#### Custom Test Helpers
```python
# tests/helpers.py
from django.test import Client
from django.contrib.auth.models import User

class AuthenticatedClient(Client):
    """Client that automatically logs in a test user"""
    
    def __init__(self, user=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if user:
            self.force_login(user)
        else:
            # Create and login default test user
            user = User.objects.create_user('testuser', 'test@example.com', 'password')
            self.force_login(user)

def create_test_user(**kwargs):
    """Helper to create test users with defaults"""
    defaults = {
        'username': 'testuser',
        'email': 'test@example.com',
        'password': 'password123',
        'is_active': True
    }
    defaults.update(kwargs)
    return User.objects.create_user(**defaults)

def assert_contains_text(response, text):
    """Assert that response contains specific text"""
    content = response.content.decode('utf-8')
    assert text in content, f"Text '{text}' not found in response"

# Using test helpers
class HelperTestExample(TestCase):
    def setUp(self):
        self.user = create_test_user()
        self.client = AuthenticatedClient(self.user)
    
    def test_view_with_authenticated_client(self):
        """Test view using authenticated client helper"""
        response = self.client.get(reverse('profile'))
        self.assertEqual(response.status_code, 200)
        assert_contains_text(response, self.user.username)
```

### Test Configuration

#### Test Settings
```python
# test_settings.py
from .settings import *

# Test-specific settings
SECRET_KEY = 'test-secret-key-not-for-production'
DEBUG = False

# Use faster database for tests
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': ':memory:',
    }
}

# Use in-memory email backend
EMAIL_BACKEND = 'django.core.mail.backends.locmem.EmailBackend'

# Disable password hashing for faster tests
PASSWORD_HASHERS = [
    'django.contrib.auth.hashers.MD5PasswordHasher',
]

# Use fast cache backend
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
    }
}

# Disable logging for tests
LOGGING = {
    'version': 1,
    'disable_existing_loggers': True,
}
```

#### Test Runner Configuration
```python
# tests/runner.py
import unittest
from django.test.runner import DiscoverRunner
from django.conf import settings

class CustomTestRunner(DiscoverRunner):
    """Custom test runner with additional features"""
    
    def setup_test_environment(self, **kwargs):
        super().setup_test_environment(**kwargs)
        # Custom test environment setup
        print("Setting up test environment...")
    
    def teardown_test_environment(self, **kwargs):
        # Custom test environment teardown
        print("Tearing down test environment...")
        super().teardown_test_environment(**kwargs)
    
    def run_tests(self, test_labels, extra_tests=None, **kwargs):
        """Run tests with custom logic"""
        # Pre-test setup
        self.pre_test_setup()
        
        # Run tests
        result = super().run_tests(test_labels, extra_tests, **kwargs)
        
        # Post-test cleanup
        self.post_test_cleanup()
        
        return result
    
    def pre_test_setup(self):
        """Setup before running tests"""
        pass
    
    def post_test_cleanup(self):
        """Cleanup after running tests"""
        pass
```

### Test Documentation

#### Test Documentation Standards
```python
class WellDocumentedTest(TestCase):
    """Example of well-documented test class.
    
    This test class demonstrates best practices for test documentation
    and organization. Each test method has a clear docstring explaining
    what it tests and why.
    """
    
    def setUp(self):
        """Set up test fixtures shared across test methods."""
        self.user = User.objects.create_user('testuser', 'test@example.com')
    
    def test_user_creation_success(self):
        """Test that user creation succeeds with valid data.
        
        This test verifies that:
        - A user can be created with valid credentials
        - The user object has the correct attributes
        - The user is active by default
        """
        self.assertEqual(self.user.username, 'testuser')
        self.assertEqual(self.user.email, 'test@example.com')
        self.assertTrue(self.user.is_active)
    
    def test_user_creation_failure_invalid_email(self):
        """Test that user creation fails with invalid email.
        
        Verifies email validation during user creation.
        """
        with self.assertRaises(ValueError):
            User.objects.create_user('user2', 'invalid-email')
```

### Test Coverage

#### Measuring Test Coverage
```bash
# Install coverage
pip install coverage

# Run tests with coverage
coverage run manage.py test

# Generate coverage report
coverage report

# Generate HTML coverage report
coverage html

# Run coverage with specific source directories
coverage run --source=myapp,utils manage.py test
```

#### Coverage Configuration
```ini
# .coveragerc
[run]
source = myapp, utils
omit =
    */migrations/*
    */tests/*
    */venv/*

[report]
exclude_lines =
    pragma: no cover
    def __repr__
    raise AssertionError
    raise NotImplementedError
    if __name__ == .__main__.:

[html]
directory = htmlcov
```

### Continuous Integration

#### GitHub Actions CI Configuration
```yaml
# .github/workflows/ci.yml
name: CI

on: [push, pull_request]

jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.8, 3.9, '3.10']
        django-version: ['3.2', '4.0']
    
    steps:
    - uses: actions/checkout@v2
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v2
      with:
        python-version: ${{ matrix.python-version }}
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
        pip install django==${{ matrix.django-version }}
    
    - name: Run tests
      run: |
        python manage.py test --verbosity=2
        coverage run manage.py test
        coverage report --fail-under=80
```

#### Pre-commit Hooks
```yaml
# .pre-commit-config.yaml
repos:
- repo: https://github.com/pre-commit/pre-commit-hooks
  rev: v4.0.1
  hooks:
  - id: trailing-whitespace
  - id: end-of-file-fixer
  - id: check-yaml
  - id: check-added-large-files

- repo: https://github.com/psf/black
  rev: 21.12b0
  hooks:
  - id: black

- repo: https://github.com/pycqa/flake8
  rev: 4.0.1
  hooks:
  - id: flake8

- repo: local
  hooks:
  - id: django-tests
    name: Run Django tests
    entry: python manage.py test
    language: system
    pass_filenames: false
    always_run: true
```

### Test Performance Monitoring

#### Test Timing and Profiling
```python
# tests/performance.py
import time
from functools import wraps
from django.test import TestCase

def time_test(func):
    """Decorator to time test execution"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        execution_time = end_time - start_time
        print(f"{func.__name__} took {execution_time:.4f} seconds")
        
        # Log slow tests
        if execution_time > 1.0:
            print(f"WARNING: {func.__name__} is slow ({execution_time:.2f}s)")
        
        return result
    return wrapper

class PerformanceTest(TestCase):
    @time_test
    def test_slow_operation(self):
        """Test that might be slow"""
        # Simulate slow operation
        time.sleep(0.5)
        self.assertTrue(True)
    
    def test_query_performance(self):
        """Test database query performance"""
        with self.assertNumQueries(1):
            # Operation that should make exactly 1 query
            Article.objects.select_related('author').filter(status='published')
```

### Test Maintenance

#### Keeping Tests Maintainable
```python
# tests/maintenance.py
from django.test import TestCase
from django.db import connection
import os

class TestMaintenanceTest(TestCase):
    """Tests to help maintain the test suite"""
    
    def test_no_broken_fixtures(self):
        """Test that all fixture files are valid JSON"""
        fixtures_dir = os.path.join(os.path.dirname(__file__), 'fixtures')
        
        for filename in os.listdir(fixtures_dir):
            if filename.endswith('.json'):
                filepath = os.path.join(fixtures_dir, filename)
                with open(filepath, 'r') as f:
                    try:
                        import json
                        json.load(f)
                    except json.JSONDecodeError as e:
                        self.fail(f"Invalid JSON in fixture {filename}: {e}")
    
    def test_all_models_have_tests(self):
        """Test that all models have corresponding test files"""
        from django.apps import apps
        from myapp.models import *  # Import all models
        
        model_names = [model.__name__.lower() for model in apps.get_models()]
        test_files = [f for f in os.listdir(os.path.dirname(__file__)) 
                     if f.startswith('test_') and f.endswith('.py')]
        
        # This is a basic check - you might want more sophisticated logic
        for model_name in model_names:
            test_file = f'test_{model_name}s.py'
            if test_file not in test_files:
                print(f"Warning: No test file found for {model_name} model")
```

### Best Practices Summary

#### Test Organization
- Group related tests in classes
- Use descriptive test method names
- Keep tests focused and isolated
- Use setUp and tearDown appropriately

#### Test Quality
- Test behavior, not implementation
- Cover happy path and edge cases
- Use appropriate assertions
- Keep tests fast and reliable

#### Test Maintenance
- Refactor tests when code changes
- Remove obsolete tests
- Keep test data realistic
- Document complex test scenarios

#### CI/CD Integration
- Run tests on every commit
- Monitor test performance
- Fail builds on test failures
- Generate coverage reports

## Performance Testing

Testing application performance and identifying bottlenecks.

### Load Testing

#### Using Locust for Load Testing
```python
# locustfile.py
from locust import HttpUser, task, between
import json

class WebsiteUser(HttpUser):
    wait_time = between(1, 5)
    
    @task
    def view_homepage(self):
        self.client.get("/")
    
    @task(3)
    def view_articles(self):
        self.client.get("/articles/")
    
    @task(2)
    def create_article(self):
        self.client.post("/articles/create/", {
            "title": "Load Test Article",
            "content": "Test content",
            "status": "draft"
        })
    
    @task
    def api_articles(self):
        self.client.get("/api/articles/")

# Run load test
# locust -f locustfile.py --host=http://localhost:8000
```

### Django Testing Tools

#### Query Counting
```python
# tests/test_performance.py
from django.test import TestCase
from django.test.utils import override_settings
from django.db import connection

class QueryPerformanceTest(TestCase):
    def test_article_list_query_count(self):
        """Test that article list view makes reasonable number of queries"""
        # Create test data
        for i in range(10):
            Article.objects.create(
                title=f"Article {i}",
                content=f"Content {i}",
                status="published"
            )
        
        # Test with query counting
        with self.assertNumQueries(3):  # Adjust expected count
            response = self.client.get(reverse('article_list'))
            self.assertEqual(response.status_code, 200)
    
    def test_select_related_usage(self):
        """Test that select_related reduces queries"""
        author = Author.objects.create(name="Test Author")
        articles = [
            Article.objects.create(title=f"Article {i}", author=author)
            for i in range(5)
        ]
        
        # Without select_related - N+1 queries
        with self.assertNumQueries(1 + len(articles)):
            for article in Article.objects.all():
                _ = article.author.name
        
        # With select_related - 1 query
        with self.assertNumQueries(1):
            for article in Article.objects.select_related('author').all():
                _ = article.author.name
```

#### Response Time Testing
```python
import time

class ResponseTimeTest(TestCase):
    def test_view_response_time(self):
        """Test that views respond within acceptable time"""
        start_time = time.time()
        response = self.client.get(reverse('article_list'))
        end_time = time.time()
        
        response_time = end_time - start_time
        self.assertLess(response_time, 1.0)  # Less than 1 second
        self.assertEqual(response.status_code, 200)
    
    def test_api_response_time(self):
        """Test API response times"""
        import json
        
        # Test multiple requests
        times = []
        for _ in range(10):
            start = time.time()
            response = self.client.get('/api/articles/')
            end = time.time()
            times.append(end - start)
            self.assertEqual(response.status_code, 200)
        
        avg_time = sum(times) / len(times)
        max_time = max(times)
        
        # Assert performance requirements
        self.assertLess(avg_time, 0.5)  # Average under 500ms
        self.assertLess(max_time, 2.0)  # Max under 2 seconds
```

### Profiling Tests

#### Using Django Debug Toolbar in Tests
```python
# tests/test_debug_toolbar.py
from django.test import TestCase
from django.test.utils import override_settings

@override_settings(
    DEBUG=True,
    INSTALLED_APPS=[
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'debug_toolbar',
    ],
    MIDDLEWARE=[
        'debug_toolbar.middleware.DebugToolbarMiddleware',
        # ... other middleware
    ]
)
class DebugToolbarTest(TestCase):
    def test_sql_queries_count(self):
        """Test SQL query count using debug toolbar data"""
        # This requires custom middleware to capture debug data
        response = self.client.get(reverse('article_list'))
        
        # Check that debug data is available
        self.assertIn('debug_toolbar', response.context)
        
        # Access debug data (requires custom setup)
        debug_data = response.context['debug_toolbar']
        sql_queries = debug_data.get('sql', [])
        
        # Assert reasonable query count
        self.assertLess(len(sql_queries), 10)
```

### Memory Usage Testing

#### Testing Memory Usage
```python
# tests/test_memory.py
import psutil
import os
from django.test import TestCase

class MemoryUsageTest(TestCase):
    def test_view_memory_usage(self):
        """Test memory usage of a view"""
        process = psutil.Process(os.getpid())
        initial_memory = process.memory_info().rss / 1024 / 1024  # MB
        
        # Perform operation that might use memory
        for _ in range(100):
            Article.objects.create(
                title=f"Memory Test Article {_}",
                content="Test content" * 100  # Large content
            )
        
        final_memory = process.memory_info().rss / 1024 / 1024
        memory_used = final_memory - initial_memory
        
        # Assert memory usage is reasonable
        self.assertLess(memory_used, 50)  # Less than 50MB increase
    
    def test_queryset_memory_efficiency(self):
        """Test that querysets don't load everything into memory"""
        # Create many objects
        for i in range(1000):
            Article.objects.create(title=f"Article {i}", content="Content")
        
        # Test iterator usage (memory efficient)
        titles = []
        for article in Article.objects.all().iterator():
            titles.append(article.title)
        
        self.assertEqual(len(titles), 1000)
        
        # Compare memory usage with and without iterator
        # (This would require more sophisticated memory tracking)
```

### Stress Testing

#### Database Stress Testing
```python
# tests/test_stress.py
from django.test import TransactionTestCase
from django.db import connection
import threading
import time

class DatabaseStressTest(TransactionTestCase):
    def test_concurrent_writes(self):
        """Test database performance under concurrent writes"""
        def create_articles(n):
            for i in range(n):
                Article.objects.create(
                    title=f"Concurrent Article {threading.current_thread().name} {i}",
                    content="Test content"
                )
        
        # Create multiple threads
        threads = []
        for i in range(5):
            thread = threading.Thread(target=create_articles, args=(20,))
            threads.append(thread)
        
        # Start all threads
        start_time = time.time()
        for thread in threads:
            thread.start()
        
        # Wait for completion
        for thread in threads:
            thread.join()
        end_time = time.time()
        
        # Check results
        total_articles = Article.objects.count()
        self.assertEqual(total_articles, 100)  # 5 threads * 20 articles
        
        # Check performance
        duration = end_time - start_time
        self.assertLess(duration, 10)  # Should complete within 10 seconds
```

### Caching Performance Tests

#### Cache Performance Testing
```python
# tests/test_cache_performance.py
from django.core.cache import cache
from django.test import TestCase
import time

class CachePerformanceTest(TestCase):
    def setUp(self):
        # Clear cache before each test
        cache.clear()
    
    def test_cache_hit_performance(self):
        """Test cache hit performance"""
        # Set cache value
        cache.set('test_key', 'test_value', 300)
        
        # Measure cache hit time
        start_time = time.time()
        for _ in range(1000):
            value = cache.get('test_key')
            self.assertEqual(value, 'test_value')
        end_time = time.time()
        
        avg_time = (end_time - start_time) / 1000
        self.assertLess(avg_time, 0.001)  # Less than 1ms per operation
    
    def test_cache_miss_performance(self):
        """Test cache miss performance"""
        start_time = time.time()
        for _ in range(1000):
            value = cache.get('nonexistent_key')
            self.assertIsNone(value)
        end_time = time.time()
        
        avg_time = (end_time - start_time) / 1000
        self.assertLess(avg_time, 0.001)
    
    def test_cached_view_performance(self):
        """Test performance improvement with view caching"""
        # Create test data
        for i in range(100):
            Article.objects.create(title=f"Article {i}", content="Content")
        
        # Time uncached request
        start_time = time.time()
        response1 = self.client.get(reverse('article_list'))
        uncached_time = time.time() - start_time
        
        # Time cached request
        start_time = time.time()
        response2 = self.client.get(reverse('article_list'))
        cached_time = time.time() - start_time
        
        # Cached request should be faster
        self.assertLess(cached_time, uncached_time)
        self.assertEqual(response1.status_code, 200)
        self.assertEqual(response2.status_code, 200)
```

### API Performance Testing

#### REST API Performance
```python
# tests/test_api_performance.py
from rest_framework.test import APITestCase
from rest_framework import status
import time
import json

class APIPerformanceTest(APITestCase):
    def setUp(self):
        # Create test data
        for i in range(100):
            Article.objects.create(
                title=f"API Article {i}",
                content=f"Content {i}",
                status="published"
            )
    
    def test_list_endpoint_performance(self):
        """Test API list endpoint performance"""
        start_time = time.time()
        response = self.client.get('/api/articles/')
        end_time = time.time()
        
        self.assertEqual(response.status_code, status.HTTP_200_OK)
        self.assertLess(end_time - start_time, 1.0)  # Under 1 second
        
        # Check pagination
        data = response.data
        self.assertIn('results', data)
        self.assertEqual(len(data['results']), 10)  # Default page size
    
    def test_bulk_operations_performance(self):
        """Test bulk create/update operations"""
        bulk_data = [
            {
                'title': f'Bulk Article {i}',
                'content': f'Content {i}',
                'status': 'draft'
            }
            for i in range(50)
        ]
        
        start_time = time.time()
        response = self.client.post(
            '/api/articles/bulk_create/',
            json.dumps(bulk_data),
            content_type='application/json'
        )
        end_time = time.time()
        
        self.assertEqual(response.status_code, status.HTTP_201_CREATED)
        self.assertLess(end_time - start_time, 5.0)  # Under 5 seconds for 50 items
```

### Continuous Performance Monitoring

#### Performance Regression Tests
```python
# tests/test_performance_regression.py
from django.test import TestCase
import time
import json

class PerformanceRegressionTest(TestCase):
    """Tests to catch performance regressions"""
    
    # Performance baselines (adjust based on your application)
    MAX_QUERY_TIME = 0.1  # 100ms
    MAX_RESPONSE_TIME = 1.0  # 1 second
    MAX_MEMORY_INCREASE = 10  # 10MB
    
    def test_homepage_performance(self):
        """Test homepage load performance"""
        start_time = time.time()
        response = self.client.get('/')
        end_time = time.time()
        
        response_time = end_time - start_time
        
        # Assert performance requirements
        self.assertLess(response_time, self.MAX_RESPONSE_TIME)
        self.assertEqual(response.status_code, 200)
        
        # Log for monitoring
        print(f"Homepage response time: {response_time:.4f}s")
    
    def test_database_query_performance(self):
        """Test database query performance baselines"""
        from django.db import connection
        
        # Test a typical query
        start_time = time.time()
        articles = list(Article.objects.filter(status='published')[:10])
        end_time = time.time()
        
        query_time = end_time - start_time
        query_count = len(connection.queries)
        
        self.assertLess(query_time, self.MAX_QUERY_TIME)
        self.assertLessEqual(query_count, 2)  # Should be 1-2 queries max
        
        print(f"Query time: {query_time:.4f}s, Queries: {query_count}")
    
    def test_api_endpoint_performance(self):
        """Test API endpoint performance"""
        response_times = []
        
        # Test multiple requests
        for _ in range(10):
            start_time = time.time()
            response = self.client.get('/api/articles/')
            end_time = time.time()
            
            response_times.append(end_time - start_time)
            self.assertEqual(response.status_code, 200)
        
        avg_time = sum(response_times) / len(response_times)
        max_time = max(response_times)
        
        self.assertLess(avg_time, 0.5)  # Average under 500ms
        self.assertLess(max_time, self.MAX_RESPONSE_TIME)
        
        print(f"API avg time: {avg_time:.4f}s, max time: {max_time:.4f}s")
```

### Performance Testing Best Practices

#### Setting Performance Baselines
```python
# performance_baselines.py
PERFORMANCE_BASELINES = {
    'homepage_load_time': 0.5,  # seconds
    'api_response_time': 0.2,   # seconds
    'database_query_time': 0.05, # seconds
    'max_memory_usage': 50,     # MB
    'max_concurrent_users': 1000,
    'max_response_time_95th_percentile': 2.0,  # seconds
}

# tests/test_performance_baselines.py
from django.test import TestCase
from myapp.performance_baselines import PERFORMANCE_BASELINES
import time

class PerformanceBaselineTest(TestCase):
    """Tests that enforce performance baselines"""
    
    def test_homepage_baseline(self):
        """Test homepage meets performance baseline"""
        start_time = time.time()
        response = self.client.get('/')
        end_time = time.time()
        
        response_time = end_time - start_time
        baseline = PERFORMANCE_BASELINES['homepage_load_time']
        
        self.assertLess(
            response_time, 
            baseline, 
            f"Homepage load time {response_time:.4f}s exceeds baseline {baseline}s"
        )
    
    def test_api_baseline(self):
        """Test API meets performance baseline"""
        start_time = time.time()
        response = self.client.get('/api/articles/')
        end_time = time.time()
        
        response_time = end_time - start_time
        baseline = PERFORMANCE_BASELINES['api_response_time']
        
        self.assertLess(
            response_time,
            baseline,
            f"API response time {response_time:.4f}s exceeds baseline {baseline}s"
        )
```

#### Automated Performance Reporting
```python
# performance_report.py
import time
import psutil
import os
from django.db import connection
from django.test.utils import get_runner
from django.conf import settings

class PerformanceReporter:
    """Generate performance reports during testing"""
    
    def __init__(self):
        self.start_time = None
        self.start_memory = None
        self.start_queries = None
    
    def start_measurement(self):
        """Start performance measurement"""
        self.start_time = time.time()
        self.start_memory = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
        self.start_queries = len(connection.queries)
    
    def end_measurement(self):
        """End performance measurement and return report"""
        end_time = time.time()
        end_memory = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
        end_queries = len(connection.queries)
        
        return {
            'duration': end_time - self.start_time,
            'memory_used': end_memory - self.start_memory,
            'queries_executed': end_queries - self.start_queries,
        }
    
    def report_performance(self, test_name, metrics):
        """Report performance metrics"""
        print(f"\nPerformance Report for {test_name}:")
        print(f"  Duration: {metrics['duration']:.4f}s")
        print(f"  Memory Used: {metrics['memory_used']:.2f}MB")
        print(f"  Queries: {metrics['queries_executed']}")
        
        # Check against baselines
        if metrics['duration'] > 1.0:
            print(f"  WARNING: Test took longer than 1 second")
        if metrics['memory_used'] > 10:
            print(f"  WARNING: Test used more than 10MB of memory")
        if metrics['queries_executed'] > 10:
            print(f"  WARNING: Test executed more than 10 queries")

# Usage in tests
class PerformanceReportedTest(TestCase):
    def setUp(self):
        self.performance_reporter = PerformanceReporter()
    
    def test_with_performance_reporting(self):
        """Test with automatic performance reporting"""
        self.performance_reporter.start_measurement()
        
        # Your test code here
        response = self.client.get('/')
        self.assertEqual(response.status_code, 200)
        
        # Generate performance report
        metrics = self.performance_reporter.end_measurement()
        self.performance_reporter.report_performance(
            'test_with_performance_reporting',
            metrics
        )
```

### Performance Testing Tools

#### Django Silk for Profiling
```python
# settings.py
INSTALLED_APPS = [
    'silk',
]

MIDDLEWARE = [
    'silk.middleware.SilkyMiddleware',
    # ... other middleware
]

# tests/test_silk_profiling.py
from django.test import TestCase
from silk.models import Request

class SilkProfilingTest(TestCase):
    def test_request_profiling(self):
        """Test request profiling with Silk"""
        # Clear existing requests
        Request.objects.all().delete()
        
        # Make request
        response = self.client.get('/')
        self.assertEqual(response.status_code, 200)
        
        # Check profiling data
        request_obj = Request.objects.first()
        self.assertIsNotNone(request_obj)
        
        # Check query count
        self.assertLess(request_obj.num_sql_queries, 20)
        
        # Check response time
        self.assertLess(request_obj.time_taken, 1.0)
```

### Best Practices for Performance Testing

#### 1. Set Realistic Baselines
- Measure performance on production-like data
- Account for caching effects
- Consider network latency
- Test under various loads

#### 2. Test Different Scenarios
- Empty database vs populated database
- Cached vs uncached requests
- Peak load vs normal load
- Different user types and permissions

#### 3. Monitor Key Metrics
- Response time (average, 95th percentile, max)
- Database query count and execution time
- Memory usage
- CPU usage
- Error rates

#### 4. Automate Performance Tests
- Run performance tests in CI/CD
- Set up alerts for performance regressions
- Generate performance reports
- Compare against historical data

#### 5. Performance Testing Tools
- **Locust**: Load testing
- **Django Debug Toolbar**: Development profiling
- **Silk**: Production profiling
- **New Relic**: Application monitoring
- **Apache Bench**: HTTP load testing
- **wrk**: Modern HTTP benchmarking

## Testing Best Practices and Common Patterns

Comprehensive guide to Django testing best practices.

### Test Structure and Organization

#### Test File Organization
```python
# tests/__init__.py
# Empty file to make tests a package

# tests/test_models.py
from django.test import TestCase
from myapp.models import Article

class ArticleModelTest(TestCase):
    """Test Article model behavior"""
    pass

# tests/test_views.py
class ArticleViewTest(TestCase):
    """Test Article views"""
    pass

# tests/test_forms.py
class ArticleFormTest(TestCase):
    """Test Article forms"""
    pass

# tests/test_api.py
from rest_framework.test import APITestCase

class ArticleAPITest(APITestCase):
    """Test Article API endpoints"""
    pass
```

### Test Naming Conventions

#### Good Test Names
```python
class ArticleTest(TestCase):
    # ✓ Good: Descriptive and specific
    def test_creation_succeeds_with_valid_data(self):
        pass
    
    def test_creation_fails_with_missing_title(self):
        pass
    
    def test_publish_method_changes_status_to_published(self):
        pass
    
    def test_list_view_shows_only_published_articles(self):
        pass
    
    # ✗ Bad: Not descriptive
    def test1(self):
        pass
    
    def test_article(self):
        pass
```

### Test Data Management

#### Using setUp and setUpTestData
```python
class ArticleTest(TestCase):
    @classmethod
    def setUpTestData(cls):
        """Set up data for entire test class (runs once)"""
        cls.author = Author.objects.create(name="Test Author")
        cls.articles = Article.objects.bulk_create([
            Article(title=f"Article {i}", author=cls.author)
            for i in range(10)
        ])
    
    def setUp(self):
        """Set up data for each test method"""
        self.draft_article = Article.objects.create(
            title="Draft Article",
            status="draft",
            author=self.author
        )
    
    def test_published_articles_count(self):
        """Test counting published articles"""
        published_count = Article.objects.filter(status='published').count()
        self.assertEqual(published_count, 10)  # From setUpTestData
    
    def test_draft_article_exists(self):
        """Test that draft article from setUp exists"""
        self.assertTrue(
            Article.objects.filter(title="Draft Article").exists()
        )
```

### Assertions and Testing Patterns

#### Common Assertion Patterns
```python
class AssertionExamplesTest(TestCase):
    def test_assertion_patterns(self):
        """Examples of common assertion patterns"""
        # Equality
        self.assertEqual(2 + 2, 4)
        self.assertNotEqual(2 + 2, 5)
        
        # Truthiness
        self.assertTrue(True)
        self.assertFalse(False)
        self.assertIsNone(None)
        self.assertIsNotNone("value")
        
        # Collections
        self.assertIn("item", ["item", "other"])
        self.assertNotIn("missing", ["item", "other"])
        self.assertEqual(len([1, 2, 3]), 3)
        
        # Exceptions
        with self.assertRaises(ValueError):
            int("not_a_number")
        
        with self.assertRaisesMessage(ValueError, "invalid literal"):
            int("not_a_number")
        
        # Django-specific
        response = self.client.get("/")
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Welcome")
        self.assertTemplateUsed(response, "base.html")
        
        # Database
        self.assertEqual(Article.objects.count(), 5)
        self.assertTrue(Article.objects.filter(title="Test").exists())
```

### Testing Edge Cases and Error Conditions

#### Testing Error Scenarios
```python
class ErrorHandlingTest(TestCase):
    def test_invalid_form_submission(self):
        """Test form validation errors"""
        response = self.client.post("/articles/create/", {
            "title": "",  # Invalid: empty title
            "content": "Valid content"
        })
        
        self.assertEqual(response.status_code, 200)  # Form re-displayed
        self.assertContains(response, "This field is required")
        self.assertFormError(response, "form", "title", "This field is required")
    
    def test_404_for_nonexistent_article(self):
        """Test 404 for non-existent resources"""
        response = self.client.get("/articles/999/")
        self.assertEqual(response.status_code, 404)
    
    def test_permission_denied(self):
        """Test permission restrictions"""
        # Try to access admin without login
        response = self.client.get("/admin/")
        self.assertRedirects(response, "/admin/login/?next=/admin/")
    
    def test_database_integrity_errors(self):
        """Test database constraint violations"""
        from django.db import IntegrityError
        
        # Try to create duplicate unique field
        Article.objects.create(title="Unique Title", slug="unique-slug")
        
        with self.assertRaises(IntegrityError):
            Article.objects.create(title="Another Title", slug="unique-slug")
```

### Testing Authentication and Authorization

#### Authentication Testing
```python
from django.contrib.auth.models import User

class AuthenticationTest(TestCase):
    def setUp(self):
        self.user = User.objects.create_user(
            username='testuser',
            password='testpass123'
        )
        self.client = Client()
    
    def test_login_success(self):
        """Test successful login"""
        response = self.client.post('/accounts/login/', {
            'username': 'testuser',
            'password': 'testpass123',
        })
        self.assertRedirects(response, '/')
        
        # Check user is logged in
        user = auth.get_user(self.client)
        self.assertTrue(user.is_authenticated)
    
    def test_login_failure(self):
        """Test login with wrong credentials"""
        response = self.client.post('/accounts/login/', {
            'username': 'testuser',
            'password': 'wrongpassword',
        })
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Invalid credentials")
    
    def test_logout(self):
        """Test user logout"""
        self.client.login(username='testuser', password='testpass123')
        response = self.client.get('/accounts/logout/')
        self.assertRedirects(response, '/')
        
        # Check user is logged out
        user = auth.get_user(self.client)
        self.assertFalse(user.is_authenticated)
```

### Testing Forms

#### Form Testing Patterns
```python
from myapp.forms import ArticleForm

class ArticleFormTest(TestCase):
    def test_form_valid_data(self):
        """Test form with valid data"""
        form_data = {
            'title': 'Valid Title',
            'content': 'Valid content',
            'status': 'draft'
        }
        form = ArticleForm(data=form_data)
        
        self.assertTrue(form.is_valid())
        article = form.save()
        self.assertEqual(article.title, 'Valid Title')
    
    def test_form_invalid_data(self):
        """Test form with invalid data"""
        form_data = {
            'title': '',  # Invalid
            'content': 'Valid content',
            'status': 'invalid_status'  # Invalid choice
        }
        form = ArticleForm(data=form_data)
        
        self.assertFalse(form.is_valid())
        self.assertIn('title', form.errors)
        self.assertIn('status', form.errors)
    
    def test_form_field_validation(self):
        """Test individual field validation"""
        form = ArticleForm()
        
        # Test title field
        title_field = form.fields['title']
        self.assertTrue(title_field.required)
        self.assertEqual(title_field.max_length, 200)
    
    def test_form_save_method(self):
        """Test form save behavior"""
        form_data = {
            'title': 'Test Article',
            'content': 'Test content',
            'status': 'published'
        }
        form = ArticleForm(data=form_data)
        
        self.assertTrue(form.is_valid())
        article = form.save()
        
        # Check that article was saved with correct data
        self.assertEqual(article.status, 'published')
        self.assertIsNotNone(article.pk)
```

### Testing Views

#### View Testing Patterns
```python
class ArticleViewTest(TestCase):
    def setUp(self):
        self.client = Client()
        self.user = User.objects.create_user('testuser', 'test@example.com')
    
    def test_list_view_get(self):
        """Test article list view GET request"""
        response = self.client.get(reverse('article_list'))
        
        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, 'articles/article_list.html')
        self.assertIn('articles', response.context)
    
    def test_detail_view_get(self):
        """Test article detail view GET request"""
        article = Article.objects.create(
            title='Test Article',
            content='Test content'
        )
        
        response = self.client.get(
            reverse('article_detail', kwargs={'pk': article.pk})
        )
        
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.context['article'], article)
    
    def test_create_view_post_valid(self):
        """Test article create view with valid POST data"""
        self.client.login(username='testuser', password='testpass123')
        
        response = self.client.post(
            reverse('article_create'),
            {
                'title': 'New Article',
                'content': 'New content',
                'status': 'draft'
            }
        )
        
        self.assertRedirects(response, reverse('article_list'))
        self.assertTrue(
            Article.objects.filter(title='New Article').exists()
        )
    
    def test_create_view_post_invalid(self):
        """Test article create view with invalid POST data"""
        self.client.login(username='testuser', password='testpass123')
        
        response = self.client.post(
            reverse('article_create'),
            {
                'title': '',  # Invalid
                'content': 'Content'
            }
        )
        
        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, 'articles/article_form.html')
        self.assertFalse(Article.objects.filter(title='').exists())
```

### Testing Models

#### Model Testing Patterns
```python
class ArticleModelTest(TestCase):
    def setUp(self):
        self.author = Author.objects.create(name="Test Author")
    
    def test_model_creation(self):
        """Test basic model creation"""
        article = Article.objects.create(
            title="Test Article",
            content="Test content",
            author=self.author
        )
        
        self.assertEqual(article.title, "Test Article")
        self.assertEqual(article.slug, "test-article")  # Auto-generated
        self.assertIsNotNone(article.pk)
    
    def test_model_str_method(self):
        """Test model string representation"""
        article = Article.objects.create(
            title="Test Article",
            author=self.author
        )
        
        expected = f"Test Article by {self.author.name}"
        self.assertEqual(str(article), expected)
    
    def test_model_methods(self):
        """Test custom model methods"""
        article = Article.objects.create(
            title="Test Article",
            content="Word " * 100,  # 100 words
            author=self.author,
            status="published"
        )
        
        self.assertTrue(article.is_published())
        self.assertEqual(article.get_word_count(), 100)
        self.assertEqual(article.get_reading_time(), 1)  # 100 words / 200 wpm
    
    def test_model_validation(self):
        """Test model field validation"""
        # Test max length
        with self.assertRaises(ValidationError):
            article = Article(
                title="A" * 201,  # Exceeds max_length
                author=self.author
            )
            article.full_clean()
    
    def test_model_relationships(self):
        """Test model relationships"""
        article = Article.objects.create(
            title="Test Article",
            author=self.author
        )
        
        # Test forward relationship
        self.assertEqual(article.author, self.author)
        
        # Test reverse relationship
        self.assertEqual(self.author.article_set.count(), 1)
        self.assertIn(article, self.author.article_set.all())
```

### Testing Managers and QuerySets

#### Manager Testing
```python
class ArticleManagerTest(TestCase):
    def setUp(self):
        self.author = Author.objects.create(name="Test Author")
        
        # Create articles with different statuses
        self.draft = Article.objects.create(
            title="Draft Article",
            status="draft",
            author=self.author
        )
        self.published = Article.objects.create(
            title="Published Article",
            status="published",
            author=self.author,
            pub_date=timezone.now() - timezone.timedelta(days=1)
        )
        self.future = Article.objects.create(
            title="Future Article",
            status="published",
            author=self.author,
            pub_date=timezone.now() + timezone.timedelta(days=1)
        )
    
    def test_published_manager(self):
        """Test published manager returns only published articles"""
        published_articles = Article.published.all()
        
        self.assertEqual(published_articles.count(), 1)
        self.assertIn(self.published, published_articles)
        self.assertNotIn(self.draft, published_articles)
        self.assertNotIn(self.future, published_articles)
    
    def test_default_manager(self):
        """Test default manager returns all articles"""
        all_articles = Article.objects.all()
        self.assertEqual(all_articles.count(), 3)
```

### Testing Signals

#### Signal Testing
```python
from django.db.models.signals import post_save
from unittest.mock import patch

class ArticleSignalTest(TestCase):
    def test_slug_creation_signal(self):
        """Test that slug is created automatically on save"""
        article = Article.objects.create(
            title="Test Article Title",
            content="Test content"
        )
        
        article.refresh_from_db()
        self.assertEqual(article.slug, "test-article-title")
    
    @patch('myapp.models.slugify_title')
    def test_slugify_called_on_save(self, mock_slugify):
        """Test that slugify function is called during save"""
        mock_slugify.return_value = "mocked-slug"
        
        article = Article.objects.create(
            title="Test Title",
            content="Test content"
        )
        
        mock_slugify.assert_called_once_with("Test Title")
        
        article.refresh_from_db()
        self.assertEqual(article.slug, "mocked-slug")
```

### Testing Email Functionality

#### Email Testing
```python
from django.core import mail

class EmailTest(TestCase):
    def setUp(self):
        # Clear outbox before each test
        mail.outbox = []
    
    def test_welcome_email_sent(self):
        """Test that welcome email is sent on user registration"""
        user = User.objects.create_user(
            username='newuser',
            email='newuser@example.com',
            password='password123'
        )
        
        # Trigger welcome email (however your app does this)
        # send_welcome_email(user)
        
        # Check email was sent
        self.assertEqual(len(mail.outbox), 1)
        
        email = mail.outbox[0]
        self.assertEqual(email.subject, 'Welcome!')
        self.assertEqual(email.to, ['newuser@example.com'])
        self.assertIn('Welcome to our platform', email.body)
    
    def test_email_content(self):
        """Test email content and formatting"""
        # Send a test email
        mail.send_mail(
            'Test Subject',
            'Test body with {{ user }}',
            'from@example.com',
            ['to@example.com'],
            html_message='<p>Test HTML body</p>'
        )
        
        email = mail.outbox[0]
        self.assertEqual(email.subject, 'Test Subject')
        self.assertEqual(email.body, 'Test body with {{ user }}')
        self.assertEqual(email.from_email, 'from@example.com')
        self.assertEqual(email.to, ['to@example.com'])
        self.assertIn('<p>Test HTML body</p>', email.alternatives[0][0])
```

### Testing File Uploads

#### File Upload Testing
```python
from django.core.files.uploadedfile import SimpleUploadedFile
from PIL import Image
import io

class FileUploadTest(TestCase):
    def test_image_upload(self):
        """Test image file upload"""
        # Create a test image
        image = Image.new('RGB', (100, 100), color='red')
        image_io = io.BytesIO()
        image.save(image_io, format='JPEG')
        image_io.seek(0)
        
        uploaded_file = SimpleUploadedFile(
            "test_image.jpg",
            image_io.getvalue(),
            content_type="image/jpeg"
        )
        
        # Test upload via form
        response = self.client.post(
            reverse('upload_image'),
            {'image': uploaded_file},
            format='multipart'
        )
        
        self.assertEqual(response.status_code, 200)
        
        # Check file was saved
        self.assertTrue(
            ArticleImage.objects.filter(
                image__icontains='test_image'
            ).exists()
        )
    
    def test_invalid_file_type(self):
        """Test upload validation for invalid file types"""
        invalid_file = SimpleUploadedFile(
            "test.exe",
            b"fake executable content",
            content_type="application/octet-stream"
        )
        
        response = self.client.post(
            reverse('upload_image'),
            {'image': invalid_file}
        )
        
        self.assertEqual(response.status_code, 400)
        self.assertContains(response, "Invalid file type")
```

### Testing AJAX Requests

#### AJAX Testing
```python
import json

class AjaxTest(TestCase):
    def test_ajax_like_article(self):
        """Test AJAX endpoint for liking articles"""
        self.client.login(username='testuser', password='testpass123')
        article = Article.objects.create(title="Test Article")
        
        response = self.client.post(
            reverse('article_like', kwargs={'pk': article.pk}),
            HTTP_X_REQUESTED_WITH='XMLHttpRequest'  # Mark as AJAX
        )
        
        self.assertEqual(response.status_code, 200)
        
        # Parse JSON response
        data = json.loads(response.content)
        self.assertTrue(data['success'])
        self.assertEqual(data['likes_count'], 1)
    
    def test_ajax_requires_authentication(self):
        """Test that AJAX endpoints require authentication"""
        article = Article.objects.create(title="Test Article")
        
        response = self.client.post(
            reverse('article_like', kwargs={'pk': article.pk}),
            HTTP_X_REQUESTED_WITH='XMLHttpRequest'
        )
        
        self.assertEqual(response.status_code, 403)
        
        data = json.loads(response.content)
        self.assertFalse(data['success'])
        self.assertIn('login required', data['message'].lower())
```

### Testing Middleware

#### Middleware Testing
```python
class CustomMiddlewareTest(TestCase):
    def test_middleware_adds_header(self):
        """Test that custom middleware adds expected headers"""
        response = self.client.get('/')
        
        # Check that middleware added custom header
        self.assertEqual(response['X-Custom-Header'], 'expected-value')
    
    def test_middleware_processes_request(self):
        """Test middleware request processing"""
        # Test that middleware modifies request
        response = self.client.get('/test-middleware/')
        
        # Check that middleware processed the request
        self.assertContains(response, 'middleware-processed')
```

### Testing Template Tags and Filters

#### Custom Template Tag/Filter Testing
```python
from django.template import Template, Context

class TemplateTagTest(TestCase):
    def test_custom_filter(self):
        """Test custom template filter"""
        template = Template('{% load custom_filters %}{{ "hello"|shout }}')
        context = Context({})
        result = template.render(context)
        
        self.assertEqual(result, 'HELLO!!!')
    
    def test_custom_tag(self):
        """Test custom template tag"""
        template = Template('{% load custom_tags %}{% current_time "%Y-%m-%d" %}')
        context = Context({})
        result = template.render(context)
        
        # Should be today's date
        from datetime import datetime
        expected = datetime.now().strftime('%Y-%m-%d')
        self.assertEqual(result, expected)
```

### Testing Background Tasks

#### Celery Task Testing
```python
from unittest.mock import patch

class CeleryTaskTest(TestCase):
    @patch('myapp.tasks.send_email')
    def test_send_welcome_email_task(self, mock_send):
        """Test Celery task for sending welcome email"""
        from myapp.tasks import send_welcome_email
        
        # Call task
        result = send_welcome_email('test@example.com')
        
        # Check task was called
        mock_send.assert_called_once_with('test@example.com')
    
    @patch('myapp.tasks.send_welcome_email.delay')
    def test_async_task_called_on_registration(self, mock_delay):
        """Test that async task is called during user registration"""
        # Trigger user registration
        response = self.client.post('/register/', {
            'username': 'newuser',
            'email': 'new@example.com'
        })
        
        # Check async task was scheduled
        mock_delay.assert_called_once_with('new@example.com')
```

### Testing Management Commands

#### Management Command Testing
```python
from io import StringIO
from django.core.management import call_command

class ManagementCommandTest(TestCase):
    def test_import_data_command(self):
        """Test management command for importing data"""
        out = StringIO()
        
        # Call management command
        call_command('import_data', 'test_data.csv', stdout=out)
        
        # Check output
        self.assertIn('Successfully imported', out.getvalue())
        
        # Check data was imported
        self.assertTrue(Article.objects.filter(title='Imported Article').exists())
    
    def test_command_with_options(self):
        """Test command with various options"""
        out = StringIO()
        
        call_command(
            'cleanup_data',
            '--dry-run',
            '--older-than=30',
            stdout=out
        )
        
        # Check dry run didn't actually delete anything
        self.assertIn('Would delete', out.getvalue())
        self.assertIn('articles', out.getvalue())
```

### Testing with Different Settings

#### Override Settings in Tests
```python
from django.test import override_settings

class SettingsOverrideTest(TestCase):
    @override_settings(DEBUG=True)
    def test_debug_mode_behavior(self):
        """Test behavior when DEBUG is enabled"""
        from django.conf import settings
        self.assertTrue(settings.DEBUG)
        
        # Test debug-specific functionality
        response = self.client.get('/debug-info/')
        self.assertContains(response, 'Debug information')
    
    @override_settings(
        EMAIL_BACKEND='django.core.mail.backends.locmem.EmailBackend',
        CACHE_BACKEND='django.core.cache.backends.locmem.LocMemCache'
    )
    def test_email_and_cache_settings(self):
        """Test with custom email and cache settings"""
        # Test email functionality
        mail.send_mail('Test', 'Body', 'from@test.com', ['to@test.com'])
        self.assertEqual(len(mail.outbox), 1)
        
        # Test cache functionality
        from django.core.cache import cache
        cache.set('test_key', 'test_value')
        self.assertEqual(cache.get('test_key'), 'test_value')
```

### Testing Database Transactions

#### Transaction Testing
```python
from django.db import transaction
from django.test import TransactionTestCase

class TransactionTest(TransactionTestCase):
    """Use TransactionTestCase for transaction-related tests"""
    
    def test_transaction_rollback(self):
        """Test that failed transactions are rolled back"""
        initial_count = Article.objects.count()
        
        try:
            with transaction.atomic():
                Article.objects.create(title="Should not persist")
                raise Exception("Force rollback")
        except Exception:
            pass
        
        # Article should not exist due to rollback
        self.assertEqual(Article.objects.count(), initial_count)
    
    def test_nested_transactions(self):
        """Test nested transaction behavior"""
        with transaction.atomic():
            Article.objects.create(title="Outer transaction")
            
            try:
                with transaction.atomic():
                    Article.objects.create(title="Inner transaction")
                    raise Exception("Inner failure")
            except Exception:
                pass
            
            # Inner transaction failed, but outer should still commit
            # (This depends on your database backend)
        
        self.assertTrue(Article.objects.filter(title="Outer transaction").exists())
```

### Testing Internationalization

#### i18n Testing
```python
from django.utils.translation import gettext as _, activate

class InternationalizationTest(TestCase):
    def test_translated_strings(self):
        """Test that strings are properly translated"""
        # Test English
        activate('en')
        english_text = _('Welcome')
        
        # Test Spanish
        activate('es')
        spanish_text = _('Welcome')
        
        # They should be different
        self.assertNotEqual(english_text, spanish_text)
        self.assertEqual(english_text, 'Welcome')
        # self.assertEqual(spanish_text, 'Bienvenido')  # If translation exists
    
    def test_date_formatting_in_different_locales(self):
        """Test date formatting in different locales"""
        from django.utils.formats import localize
        from datetime import datetime
        
        test_date = datetime(2024, 1, 15)
        
        # Test US format
        activate('en-us')
        us_format = localize(test_date, use_l10n=True)
        
        # Test German format
        activate('de')
        german_format = localize(test_date, use_l10n=True)
        
        # Formats should be different
        self.assertNotEqual(us_format, german_format)
```

### Testing Security Features

#### Security Testing
```python
class SecurityTest(TestCase):
    def test_sql_injection_protection(self):
        """Test that SQL injection attacks are prevented"""
        malicious_input = "'; DROP TABLE articles; --"
        
        # This should not execute dangerous SQL
        articles = Article.objects.filter(title__icontains=malicious_input)
        
        # Should not crash or delete data
        self.assertIsInstance(articles, QuerySet)
    
    def test_xss_protection(self):
        """Test that XSS attacks are prevented"""
        xss_payload = '<script>alert("XSS")</script>'
        
        article = Article.objects.create(
            title="Safe Title",
            content=xss_payload
        )
        
        response = self.client.get(
            reverse('article_detail', kwargs={'pk': article.pk})
        )
        
        # XSS payload should be escaped
        self.assertContains(response, '&lt;script&gt;')
        self.assertNotContains(response, '<script>')
    
    def test_csrf_protection(self):
        """Test that CSRF protection is working"""
        self.client.login(username='testuser', password='testpass123')
        
        # POST without CSRF token should fail
        response = self.client.post(
            reverse('article_create'),
            {'title': 'Test', 'content': 'Content'}
        )
        
        # Should get CSRF failure
        self.assertEqual(response.status_code, 403)
```

### Testing Best Practices Summary

#### 1. Test Structure
- Use descriptive test method names
- Group related tests in classes
- Keep tests focused and independent
- Use setUp and tearDown appropriately

#### 2. Test Quality
- Test behavior, not implementation
- Cover happy paths and edge cases
- Use appropriate assertions
- Write tests before or alongside code

#### 3. Test Maintenance
- Keep tests fast and reliable
- Refactor tests with code changes
- Remove obsolete tests
- Document complex test scenarios

#### 4. Test Coverage
- Aim for high coverage (>80%)
- Focus on critical paths
- Use coverage tools to identify gaps
- Don't aim for 100% coverage blindly

#### 5. Test Performance
- Keep individual tests fast (<1 second)
- Use setUpTestData for expensive setup
- Mock external dependencies
- Run tests in parallel when possible

#### 6. Test Documentation
- Use docstrings to explain test purpose
- Comment complex test logic
- Name variables clearly
- Group related assertions

#### 7. CI/CD Integration
- Run tests on every commit
- Fail builds on test failures
- Monitor test performance
- Generate coverage reports

#### 8. Common Testing Patterns
- **Arrange-Act-Assert**: Set up data, perform action, check results
- **Given-When-Then**: Describe test scenarios clearly
- **Test Doubles**: Use mocks, stubs, and fakes appropriately
- **Data Builders**: Create test data fluently

#### 9. Testing Pyramid
- **Unit Tests**: Test individual functions/methods (bottom layer, most tests)
- **Integration Tests**: Test component interactions
- **End-to-End Tests**: Test complete user workflows (top layer, fewest tests)

#### 10. Test Smells to Avoid
- Tests that depend on each other
- Tests that are slow or flaky
- Tests that test implementation details
- Tests with too many assertions
- Tests that are hard to understand
- Tests that duplicate each other